In [ ]:
import numpy as np
import numpy.linalg
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
from scipy.optimize import curve_fit
from scipy.ndimage import interpolation
from statsmodels.stats.anova import AnovaRM
import time
import sys
import math
import pylab as py
from scipy.signal import savgol_filter
from scipy.optimize import least_squares
import seaborn as sns

import statannot
import scipy.stats as sci 
# from scipy.stats import norm

import pingouin as pg
from pingouin import power_rm_anova

import matplotlib.mlab as mlb
from scipy.signal import find_peaks

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (5,5)
sns.set_theme(style="whitegrid")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Data Extractor

In [ ]:
def DataExtractor(trackedObject, phase):
  
    df_interp = None
    newArrLength = 1000

    trialsMask = (df['info'] == trackedObject) & (df['Phase'] == phase) 
    trialsOfInterest = pd.unique(df[trialsMask].trialNumber)

    for i in trialsOfInterest:
    # print(df['inShotRegion'].iloc[i:])
    # if df['inShotRegion'].iloc[i] == True:

        # Set desired masks
        objMask = (df['info'] == trackedObject) & (df['Phase'] == phase) & (df['trialNumber'] == i) # & (df['inShotRegion'] == True)
        timeMask = df.loc[(df['info'] == trackedObject) & (df['Phase'] == phase) & (df['trialNumber'] == i),['tyme']]
        cueBallMask = (df['info'] == 'Cueball') & (df['Phase'] == phase) & (df['trialNumber'] == i) # This variable mask will help us determine impact time 
        targetBallMask = (df['info'] == 'TargetBall') & (df['Phase'] == phase) & (df['trialNumber'] == i)

        # Resize all variables 
        ##########################################################################

        # Extract data from desired objects (Special cases here)
        zPos_cueball = ResizeArray(df[cueBallMask].zPos, newArrLength)
        zPos_targball = ResizeArray(df[targetBallMask].zPos, newArrLength)

        xRot_interp = ResizeArray(df[objMask].xRot, newArrLength)
        yRot_interp = ResizeArray(df[objMask].yRot, newArrLength)
        zRot_interp = ResizeArray(df[objMask].zRot, newArrLength)

        xPos_interp = ResizeArray(df[objMask].xPos, newArrLength)
        yPos_interp = ResizeArray(df[objMask].yPos, newArrLength)
        zPos_interp = ResizeArray(df[objMask].zPos, newArrLength)

        time_interp = ResizeArray(timeMask.tyme, newArrLength)

        ##########################################################################

        # Compute angular velocities
        xRv = savgol_filter(xRot_interp, 75, 4)
        xRot_vel = np.gradient(xRv)
        yRv = savgol_filter(yRot_interp, 75, 4)
        yRot_vel = np.gradient(yRv)
        zRv = savgol_filter(zRot_interp, 75, 4)
        zRot_vel = np.gradient(zRv)

        # Compute linear velocities 
        x_v = savgol_filter(xPos_interp, 75, 4)
        x_vel = np.gradient(x_v)
        y_v = savgol_filter(yPos_interp, 75, 4)
        y_vel = np.gradient(y_v)
        z_v = savgol_filter(zPos_interp, 75, 4)
        z_vel = np.gradient(z_v)

        # Save data to new dataframe 
        dataList = zip(xRot_interp, yRot_interp, zRot_interp, time_interp)
        tmpDF_int = pd.DataFrame(dataList, columns=['xRot','yRot','zRot', 'time'])

        tmpDF_int.insert(0, "zPos_targBall", zPos_targball, True)
        tmpDF_int.insert(0, "zPos_cueBall", zPos_cueball, True)

        tmpDF_int.insert(0, "zAngVel", zRot_vel, True)
        tmpDF_int.insert(0, "yAngVel", yRot_vel, True)
        tmpDF_int.insert(0, "xAngVel", xRot_vel, True)

        tmpDF_int.insert(0, "zVel", z_vel, True)
        tmpDF_int.insert(0, "yVel", y_vel, True)
        tmpDF_int.insert(0, "xVel", x_vel, True)

        tmpDF_int.insert(0, "zPos", zPos_interp, True)
        tmpDF_int.insert(0, "yPos", yPos_interp, True)
        tmpDF_int.insert(0, "xPos", xPos_interp, True)

        tmpDF_int.insert(0, "Phase", phase, True)
        tmpDF_int.insert(0, "trialNum", i, True)


        if df_interp is None:
            df_interp = tmpDF_int
        else:
            df_interp = pd.concat((df_interp, tmpDF_int))

    return df_interp, trialsOfInterest

In [ ]:
#--------------- Cross-Correlation ---------------------------------------------

def CrossCorr(vel_1, vel_2, sampleFreq):    
    
    corr = np.correlate(vel_1 - np.mean(vel_1), 
                      vel_2 - np.mean(vel_2),
                      mode='full')
    
    sampleDifference = np.argmax(vel_2) - np.argmax(vel_1) # What is this 20? np.argmax(vel_2[20:]) - np.argmax(vel_1[20:])

    lag = (sampleDifference  * sampleFreq) * 1000
    
    return lag

# Metric extractor

In [ ]:
def VisualizeTrajectories(mask, mask_virt, newArrLength = 100, save_plot=False):
    
    # Figures for illustration
#     newArrLength = 400

    # Compute actual sampling rate
    timetaken = df_all[mask]['time'].values
    timetaken2 = ResizeArray(timetaken, newArrLength)
    timetaken3 = np.round(timetaken2,1)
    timetaken3 = timetaken3.tolist()

    # If 0.0 time isn't present, then use the smallest time value as the start of the trial
    try:
        indexOfStart = timetaken3.index(0.0) # indexOfStart = np.where(timetaken == 0.0)
    except Exception as e:
        print('IndexErr: ', e)
        minTimeVal = np.nanmin(timetaken3)
        indexOfStart = timetaken3.index(minTimeVal)

    # Check if devision by zero, because the last value happens to be zero and use last largest value instead
    try:
        lastMaxTimeVal = timetaken3[-1]
        samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / timetaken3[-1]), 4)
    except Exception as e:
        print('SampleErr: ', e)
        lastMaxTimeVal = np.nanmax(timetaken3[-1-10:-1])
        samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / lastMaxTimeVal), 4)

#     print('Sampling Rate: ', np.round(1.0/samplingRate))

    # Get individual velocities for real hand ---------------------------------------
    pos_x = ResizeArray(df_all[mask]['xPos'].values, newArrLength)
    pos_xf = savgol_filter(pos_x, 21, 9)
    vel_x = np.gradient(pos_xf / samplingRate)

    pos_y = ResizeArray(df_all[mask]['yPos'].values, newArrLength)
    pos_yf = savgol_filter(pos_y, 21, 9)
    vel_y = np.gradient(pos_yf / samplingRate)

    pos_z = ResizeArray(df_all[mask]['zPos'].values, newArrLength)
    pos_zf = savgol_filter(pos_z, 21, 9)
    vel_z = np.gradient(pos_zf / samplingRate)
    vel_type_1 = np.sqrt(np.power(vel_x,2) + np.power(vel_y,2) + np.power(vel_z,2))
    vel_type_1f = savgol_filter(vel_type_1, 21, 9)

    # Get individual velocities for virtual hand ---------------------------------------
    pos_xv = ResizeArray(df_all[mask_virt]['xPos'].values, newArrLength)
    pos_xfv = savgol_filter(pos_xv, 21, 9)
    vel_xv = np.gradient(pos_xfv / samplingRate)

    pos_yv = ResizeArray(df_all[mask_virt]['yPos'].values, newArrLength)
    pos_yfv = savgol_filter(pos_yv, 21, 9)
    vel_yv = np.gradient(pos_yfv / samplingRate)

    pos_zv = ResizeArray(df_all[mask_virt]['zPos'].values, newArrLength)
    pos_zfv = savgol_filter(pos_zv, 21, 9)
    vel_zv = np.gradient(pos_zfv / samplingRate)

    vel_type_1v = np.sqrt(np.power(vel_xv,2) + np.power(vel_yv,2) + np.power(vel_zv,2))
    vel_type_1fv = savgol_filter(vel_type_1v, 21, 9)

    pos_tx = df_all[mask]['xTPos'].values
    pos_ty = df_all[mask]['yTPos'].values
    pos_tz = df_all[mask]['zTPos'].values

    # Velocity around start of trial
    startIdx = int(np.round(indexOfStart*0.5))
    siMargin = int(np.round(startIdx * 0.95)) # This gurantees that the array is always long enough rather than giving an arbitrary fixed scalar value as margins
#     print('Start Index Margin: ', siMargin)

    realVel = vel_type_1f[startIdx-siMargin:startIdx+siMargin]
    virtVel = vel_type_1fv[startIdx-siMargin:startIdx+siMargin]

    # Pos
    plt.figure()
    #                     plt.subplot(1,3,1)
    plt.plot(pos_x, pos_z,'k-o', linewidth=2) # Real Trajectory 
    plt.plot(pos_xv, pos_zv,'m-o', linewidth=2) # Virtual Trajectory 
#     plt.plot((pos_x[startIdx] + pos_xv[startIdx])/2,pos_z[startIdx],'cs',linewidth=4) # Start time point
    plt.plot([pos_x[startIdx]-0.02, pos_xv[startIdx]+0.02],[pos_z[startIdx], pos_z[startIdx]],'c--',linewidth=4) # Start time point
    plt.plot(pos_tx, pos_tz,'r-o',ms=20, alpha=0.5) # Target position
    plt.legend(['Marker','Virtual','Start', 'Target'])
    plt.plot([pos_x, pos_xv], [pos_z, pos_zv],'b-',alpha=0.5) # Connection line between trajectories to indicate delays
    
    plt.title('Example Trajectory', fontsize=16.5)
    plt.xlabel('X-Axis (m)', fontsize=14)
    plt.ylabel('Z-Axis (m)', fontsize=14)
    plt.grid(False)

    if save_plot:
        plt.savefig(str(np.round(time.time())) + '_Trajectory.jpg', dpi=600, bbox_inches='tight')
    
    plt.show()

    # Vel
    plt.figure()
    #                     plt.subplot(1,3,2)
#     plt.plot(vel_type_1,'r-')
    plt.plot(vel_type_1f,'k-', linewidth=2)
    plt.plot(vel_type_1fv,'m-', linewidth=2)
    plt.plot([startIdx,startIdx],[0,2.75],'c--',linewidth=4)
    plt.title('Vel_Type_1')
    plt.legend(['Marker','Virtual','Start'])
    #         plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
    #         plt.axis('equal')
#     plt.ylim([-0.1, 1.4])

    plt.tight_layout()
    
    print('LastTime: ', lastMaxTimeVal)
    timeInSec = np.arange(0,lastMaxTimeVal, (lastMaxTimeVal/6))
    timeInSec = np.round(timeInSec, 2)
    
    plt.title('Resultant Velocity',fontsize=16.5)
    plt.xlabel('Time (s)',fontsize=14)
    plt.ylim([-0.1, 2.5])
    
    try:
        plt.xticks([0,20,40,60,80,100],timeInSec)
    except Exception as e:
        print('X-Ticks Error: ', e)
        
    plt.ylabel("Velocity $\mathregular{ms^{-1}}$",fontsize=14)
    plt.grid(False)

    if save_plot:
        plt.savefig(str(np.round(time.time())) + '_Velocity.jpg', dpi=600, bbox_inches='tight')

    plt.show()

        # Last positional data point in different colours for visualisation purposes 
#     plt.figure()
#     #                     plt.subplot(1,3,3)
#     plt.plot(pos_x[-1-5:-1], pos_z[-1-5:-1],'k-o', ms=8)
#     plt.plot(pos_xv[-1-5:-1], pos_zv[-1-5:-1],'m-o', ms=6)
#     plt.plot([pos_x[-1], pos_xv[-1]], [pos_z[-1], pos_zv[-1]], 'r-o', ms=15, alpha=0.5)

#     #         plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
#     #         plt.axis('equal')


In [ ]:
def ResizeArray(data, newSize):
    x = data
    i = newSize
    z = i / len(x)
    x_int = interpolation.zoom(x,z)

    return x_int

def AngularCorrection(data):
    outputarr = []
    for i in data:
        if i>180:
            outputarr.append(i - 270)
        else:
            outputarr.append(i + 90)

    return outputarr

def AverageCurve(data,col):
    # plt.figure()
    ydata = np.mean(data,axis=0)
    xvls = np.linspace(0,len(ydata),len(ydata)) 
    yerr = np.std(data, axis=0) / np.sqrt(np.shape(data)[0])
    plt.plot(xvls,ydata,color=col)
    plt.fill_between(xvls, ydata-yerr, ydata+yerr, alpha=0.5,color=[0.1,0.1,0.1])

# marker='s', mfc='red', mec='red', ms=5, mew=2,

def PlotErrorBars(dataX = np.tile(np.nan,10), dataY = np.tile(np.nan,10), dataZ = np.tile(np.nan,10), colorz = 'r'):
    # Clean up outliers 5x outside the mean  
    xData =  [(i * np.nan) if i > (np.nanmean(dataX) * 5.0) else i for i in dataX]
    yData =  [(i * np.nan) if i > (np.nanmean(dataY) * 5.0) else i for i in dataY]
    zData =  [(i * np.nan) if i > (np.nanmean(dataZ) * 5.0) else i for i in dataZ]

    # Compute standard errors
    x_SE  = np.std(xData, axis=0) / np.sqrt(np.shape(xData)[0])
    y_SE  = np.std(yData, axis=0) / np.sqrt(np.shape(yData)[0])
    z_SE  = np.std(zData, axis=0) / np.sqrt(np.shape(zData)[0])
    all_SE = [x_SE, y_SE, z_SE]

    # Plot data 
    plt.errorbar([0,1,2], [np.nanmean(xData),np.nanmean(yData),np.nanmean(zData)], all_SE, color = colorz, marker='s')


def PlotErrorBars2(dataX = np.tile(np.nan,10), colorz = 'r'):
    # Clean up outliers 5x outside the mean  
    xData =  [(i * np.nan) if i > (np.nanmean(dataX) * 5.0) else i for i in dataX]

    # Compute standard errors
    x_SE  = np.std(xData, axis=0) / np.sqrt(np.shape(xData)[0])

    # Plot data 
    plt.errorbar([0], np.nanmean(xData), x_SE, color = colorz, marker='s')

In [ ]:

def AverageAngPos(df_int, trials):
    xRots, yRots, zRots = [], [], []
    xPosAv, yPosAv, zPosAv = [],[],[]
    for i in trials:
        mask_1 = (df_int['trialNum'] == i) 
        xRots.append(df_int.xRot[mask_1])
        yRots.append(df_int.yRot[mask_1])
        zRots.append(df_int.zRot[mask_1])
        xPosAv.append(df_int.xPos[mask_1])
        yPosAv.append(df_int.yPos[mask_1])
        zPosAv.append(df_int.zPos[mask_1])
        
    return xRots, yRots, zRots, xPosAv, yPosAv, zPosAv

def AvAngVel(df_int, trials):
    xAngVels, yAngVels, zAngVels = [],[],[]
    for i in trials:
        mask_1 = (df_int['trialNum'] == i) 
        xAngVels.append(np.gradient(savgol_filter(df_int.xRot[mask_1], 75, 4)))
        yAngVels.append(np.gradient(savgol_filter(df_int.yRot[mask_1], 75, 4)))
        zAngVels.append(np.gradient(savgol_filter(df_int.zRot[mask_1], 75, 4)))
    return xAngVels, yAngVels, zAngVels

def AvMaxVels(df_int, trials):
  
    # cueballZpos, targetballZpos = [], []
    xVels, yVels, zVels = [],[],[]
    maxXVels, maxYVels, maxZVels = [],[],[]
    maxAngVel = []

    for i in trials:
        mask_1 = (df_int['trialNum'] == i) 

        xVel = savgol_filter(np.gradient(df_int.xPos[mask_1]), 75,4)
        xVels.append(xVel)
        yVel = savgol_filter(np.gradient(df_int.yPos[mask_1]), 75,4)
        yVels.append(yVel)
        zVel = savgol_filter(np.gradient(df_int.zPos[mask_1]), 75,4)
        zVels.append(zVel)

        maxXVels.append(np.max(xVel[100:400]))
        maxYVels.append(np.max(yVel[100:400]))
        maxZVels.append(np.max(zVel[100:400]))

        # cueballZpos.append(df_int.zPos_cueBall[mask_1])
        # targetballZpos.append(df_int.zPos_targBall[mask_1])

    return xVels, yVels, zVels, maxXVels, maxYVels, maxZVels


In [ ]:
def ExtractMetrics(df_all, tempores, heightes):

#     tempores = '80'
#     heightes = 'low'

    df_metrics = None
    np.set_printoptions(suppress=True)
    newArrLength = 100
    plotting = True
    keepPlottingFor = 10

    # mask = (df_all['PtxID'] == 'Susan') & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == 'low') & (df_all['tempos'] == '80') & (df_all['TrialNum'] == '42')
    ptxIds = pd.unique(df_all['PtxID'])
    print('Participants: ', ptxIds)
    # Ptx number: 
    # 0 = Susan
    # 1 = Davide
    # 2 = Joe # <- Exluded participant due to missing tempo labels 
    # 3 = Poppy
    # 4 = Katrina
    # 5 = Max # <- Exluded participant due to missing tempo labels 
    # 6 = Pete
    includedPtxs = [0,1,3,4,6]

    trials = np.arange(0,72)

    masktemp = (df_all['gameObjectName'] == 'realFingerTip')
    trialNumbs = pd.unique(df_all[masktemp].TrialNum)
    
    for party in includedPtxs:

        ptxNum = party

        for trial in trials:

            print('\nTrial number: ', trial)

            try:
                mask = (df_all['TrialNum'] == str(trial)) & (df_all['PtxID'] == ptxIds[ptxNum]) & (df_all['tempos'] == tempores) & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == heightes)
                mask_virt = (df_all['TrialNum'] == str(trial)) & (df_all['PtxID'] == ptxIds[ptxNum]) & (df_all['tempos'] == tempores) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['height'] == heightes)

                # Compute actual sampling rate
                timetaken = df_all[mask]['time'].values
                timetaken2 = ResizeArray(timetaken, newArrLength)
                timetaken3 = np.round(timetaken2,1)
                timetaken3 = timetaken3.tolist()

                # If 0.0 time isn't present, then use the smallest time value as the start of the trial
                try:
                    indexOfStart = timetaken3.index(0.0) # indexOfStart = np.where(timetaken == 0.0)
                except Exception as e:
                    print('IndexErr: ', e)
                    minTimeVal = np.nanmin(timetaken3)
                    indexOfStart = timetaken3.index(minTimeVal)

                # Check if devision by zero, because the last value happens to be zero and use last largest value instead
                try:
                    samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / timetaken3[-1]), 4)
                except Exception as e:
                    print('SampleErr: ', e)
                    lastMaxTimeVal = np.nanmax(timetaken3[-1-10:-1])
                    samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / lastMaxTimeVal), 4)

                print('Sampling Rate: ', np.round(1.0/samplingRate))

                # Get individual velocities for real hand ---------------------------------------
                pos_x = ResizeArray(df_all[mask]['xPos'].values, newArrLength)
                pos_xf = savgol_filter(pos_x, 21, 9)
                vel_x = np.gradient(pos_xf / samplingRate)

                pos_y = ResizeArray(df_all[mask]['yPos'].values, newArrLength)
                pos_yf = savgol_filter(pos_y, 21, 9)
                vel_y = np.gradient(pos_yf / samplingRate)

                pos_z = ResizeArray(df_all[mask]['zPos'].values, newArrLength)
                pos_zf = savgol_filter(pos_z, 21, 9)
                vel_z = np.gradient(pos_zf / samplingRate)
                vel_type_1 = np.sqrt(np.power(vel_x,2) + np.power(vel_y,2) + np.power(vel_z,2))
                vel_type_1f = savgol_filter(vel_type_1, 21, 9)

                # Get individual velocities for virtual hand ---------------------------------------
                pos_xv = ResizeArray(df_all[mask_virt]['xPos'].values, newArrLength)
                pos_xfv = savgol_filter(pos_xv, 21, 9)
                vel_xv = np.gradient(pos_xfv / samplingRate)

                pos_yv = ResizeArray(df_all[mask_virt]['yPos'].values, newArrLength)
                pos_yfv = savgol_filter(pos_yv, 21, 9)
                vel_yv = np.gradient(pos_yfv / samplingRate)

                pos_zv = ResizeArray(df_all[mask_virt]['zPos'].values, newArrLength)
                pos_zfv = savgol_filter(pos_zv, 21, 9)
                vel_zv = np.gradient(pos_zfv / samplingRate)

                vel_type_1v = np.sqrt(np.power(vel_xv,2) + np.power(vel_yv,2) + np.power(vel_zv,2))
                vel_type_1fv = savgol_filter(vel_type_1v, 21, 9)
                # ------------------------------------------------------------------------------

                pos_tx = df_all[mask]['xTPos'].values
                pos_ty = df_all[mask]['yTPos'].values
                pos_tz = df_all[mask]['zTPos'].values

                # Velocity around start of trial
                startIdx = indexOfStart
                siMargin = int(np.round(startIdx * 0.95)) # This gurantees that the array is always long enough rather than giving an arbitrary fixed scalar value as margins
                print('Start Index Margin: ', siMargin)

                realVel = vel_type_1f[startIdx-siMargin:startIdx+siMargin]
                virtVel = vel_type_1fv[startIdx-siMargin:startIdx+siMargin]

                
#                 timeMask = df_all['TrialNum'] == trialNumbs[trial]
#                 times = df_all[timeMask].time.tolist()
                
                try:    
#                     lag = CrossCorr(realVel, virtVel, samplingRate)
                    lag = CrossCorr2(realVel, virtVel, samplingRate)
                    lagings2 = np.abs(np.round(1000 * (samplingRate * lag), 5)) # sampleRate = sampleTime
                except Exception as e:
                    print('LagErr: ', e)
                    realVel = np.zeros(100)
                    virtVel = np.zeros(100)
                    lagings2 = np.nan()
                    
                
                lagings2 = np.sqrt(lagings2*lagings2)
                print('Lag: ', lagings2, ' ms')


                maxVel = np.max(realVel)
                maxVelVirt = np.max(virtVel)
                print('Max vel: \n', 'Real: ', np.round(maxVel,2),'\n', 'Virt: ', np.round(maxVelVirt,2))

                real = np.asarray([pos_x,pos_z])
                virt = np.asarray([pos_xv,pos_zv])

                manhattanErr = np.sum(np.abs(real-virt)) / len(pos_x)

                ns = 5
                p1m = [np.nanmean(pos_x[-1-ns:-1]), np.nanmean(pos_z[-1-ns:-1])]
                p2m = [np.nanmean(pos_xv[-1-ns:-1]), np.nanmean(pos_zv[-1-ns:-1])]
                distance2 = np.round(math.sqrt( ((p1m[0]-p2m[0])**2)+((p1m[1]-p2m[1])**2) ) * 100,2)

                pathOffset = np.round(manhattanErr * 100,2)
                print('Per sample path offset (Manhattan err): ', pathOffset, ' cm')
                print('End point err: ', distance2, ' cm')

                
                # Lag corrected path offset -------------------------------------------------
#                 averageLag = np.round(np.nanmean(lagings),3)
                # Absolute angular error  
                try:
                    lag2 = np.sqrt(lag*lag) # Roll back by absolute number of lag samples 
                    rollVal = int(np.round(lag2))
                    offsetDiff = np.roll(real,0) - np.roll(virt,-rollVal)
                    manhattanErr_2 = np.sum(np.abs(offsetDiff)) / len(pos_x)
                    pathOffset_2 = np.round(manhattanErr_2 * 100,2)
                except:
                    pathOffset_2 = pathOffset
                    
                # Lag corrected path offset end ---------------------------------------------
                    

                
                targetID = df_all[mask]['targetID'][0]
                lateralPos = ''
                if '1' in targetID or '2' in targetID: 
                    lateralPos = 'left'
                elif '5' in targetID or '6' in targetID:
                    lateralPos = 'right'
                else:
                    lateralPos = 'center'
                
                verticalPos = ''
                if 'A' in targetID:
                    verticalPos = 'far'
                elif 'C' in targetID:
                    verticalPos = 'close'
                else:
                    verticalPos = 'middle'
                
                data = {'PtxID' : [ptxIds[ptxNum]], 
                        'TrialNum' : [str(trial)], 
                        'TargetID' : [targetID],
                        'TargetLateral' : [lateralPos],
                        'TargetVertical' : [verticalPos],
                        'MaxVel_Real': [maxVel], 
                        'MaxVel_Virt': [maxVelVirt],
                        'Lag' : [lagings2], 
                        'PathOffset' : [pathOffset], 
                        'PathOffsetNoLag' : [pathOffset_2], 
                        'EndError' : [distance2]}

                tmpDF = pd.DataFrame.from_dict(data)

                if df_metrics is None:
                    df_metrics = tmpDF
                else:
                    df_metrics = pd.concat((df_metrics, tmpDF))

                if plotting and trial < keepPlottingFor:
                    # Pos
                    plt.figure()
#                     plt.subplot(1,3,1)
                    plt.plot(pos_x, pos_z,'k-o') # Real Trajectory 
                    plt.plot(pos_xv, pos_zv,'m-o') # Virtual Trajectory 
                    plt.plot(pos_x[startIdx],pos_z[startIdx],'cs',ms=10) # Start time point
                    plt.plot(pos_tx, pos_tz,'r-o',ms=10) # Target position
                    plt.legend(['Real Trajectory','Virtual Trajectory','Start', 'Target'])
                    plt.plot([pos_x, pos_xv], [pos_z, pos_zv],'b-',alpha=0.5) # Connection line between trajectories to indicate delays

                    # Last positional data point in different colours for visualisation purposes 
                    plt.figure()
#                     plt.subplot(1,3,3)
                    plt.plot(pos_x[-1-5:-1], pos_z[-1-5:-1],'k-o', ms=8)
                    plt.plot(pos_xv[-1-5:-1], pos_zv[-1-5:-1],'m-o', ms=6)
                    plt.plot([pos_x[-1], pos_xv[-1]], [pos_z[-1], pos_zv[-1]], 'r-o', ms=15, alpha=0.5)

            #         plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
            #         plt.axis('equal')

                    # Vel
                    plt.figure()
#                     plt.subplot(1,3,2)
                    plt.plot(vel_type_1,'r-')
                    plt.plot(vel_type_1f,'g-')
                    plt.plot(vel_type_1fv,'m-')
                    plt.plot([startIdx,startIdx],[0,2.75],'c--',linewidth=3)
                    plt.title('Vel_Type_1')
                    plt.legend(['Raw','Filtered','Filt Virt','Start'])
            #         plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
            #         plt.axis('equal')
                    plt.ylim([-0.1, 5])

                    plt.tight_layout()
                    plt.show()

            except Exception as e:
                print('Err: ', e)

    return df_metrics

In [ ]:
def ReadData(path, file, height, tempo):
    # # Define data frame variable
    df = None 

    # Load each file into the data frame 
    for i in range(len(files)): 

        if ".json" in files[i] and "height" in files[i] and "tempo" in files[i]:  
            # if "txt.json" in files[i] and "Phase" in files[i]:
            # print(files[i])

            # Extract file name info and add to the dataframe 
            fileWords = files[i].split("_")

            # Extract user ID 
            idx = fileWords.index("bpm")
            userID = fileWords[idx - 2]

            # Add trial number to data frame 
            tmpDF = pd.read_json(path + files[i])
            tmpDF.insert(0, "UserID", userID, True)

            if df is None:
                df = tmpDF
            else:
                df = pd.concat((df, tmpDF))
            
    return df

In [ ]:
def CleanUpnNorm(df_metrics_all, metric='', norm_param=1.0):
    
    from collections import Counter
    from sklearn.preprocessing import PowerTransformer
    from sklearn.model_selection import train_test_split
    from scipy.stats import boxcox
    from scipy.stats import yeojohnson
    
    colors = ["#D81B60", "#0188FF", "#FFC107", "#B7A2FF", "#000000", "#2EC5AC"]
    BINS = 30
    data = df_metrics_all[metric]

#------------------------------------------------------------------------------------------------
#-------------------------------- Identify and remove outliers ----------------------------------
#------------------------------------------------------------------------------------------------
    
    # Interquartile method    
    q25, q75 = np.percentile(data, 25), np.percentile(data, 75)
    iqr = q75 - q25
    print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))

    # calculate the outlier cutoff
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off

    # # identify and remove outliers
    data_or = []
    for x in data:
        if x >= lower and x <= upper:
            data_or.append(x)
        else:
            data_or.append(np.nan)

    print('Original array length: ', len(data_or))
    print('Array length without outliers: ', len(data_or)-np.round((data_or.count(np.nan))))
    print('Percentage of data excluded: ', np.round((data_or.count(np.nan) / len(data_or)) * 100), ' %')

#------------------------------------------------------------------------------------------------
#----------------------------- Map data to normal distribution ----------------------------------
#------------------------------------------------------------------------------------------------
    
#     yj = PowerTransformer(method="yeo-johnson")

    X = np.asarray(data_or)
    X = X.reshape(-1, 1)

    # Mapped to normal distribution 
    data_norm = yeojohnson(X, norm_param)

    plt.figure()
    plt.subplot(1,2,1)
    plt.hist(X, bins=BINS)
    plt.title('Original Data')
    plt.ylabel('Error')
    plt.ylim([0, 300])

    plt.subplot(1,2,2)
    plt.hist(data_norm, color=colors[0], bins=BINS)
    plt.title('Transformed Data')
    plt.ylim([0, 300])

    W, p = sci.shapiro(data_norm)
    print('Shap YT', ' W: ', np.round(W, 4), ' p: ', np.round(p, 10))
    
    return data_norm

In [ ]:
def ComputeLag(Angle_real, Angle_virt, times):
    
    trialDuration = int(np.round(times[-1], 3))
    
    # Lag between virtual and real angular signals 
    # Measure peaks based on average max height using a moving average 
    mH_real = []
    mH_virt = []
    lenX = len(Angle_real)
    window_size = 100 

    for i in range(len(Angle_real) - window_size + 1):
        maxHei_real = np.nanmax(Angle_real[i: i + window_size])
        maxHei_virt = np.nanmax(Angle_virt[i: i + window_size])
        mH_real.append(maxHei_real)
        mH_virt.append(maxHei_virt)

    #         print('Max Height: ', maxHei)

    meanMaxH_real = np.nanmean(mH_real)
    meanMaxH_virt = np.nanmean(mH_virt)

    pks_real, _ = find_peaks(Angle_real, height=meanMaxH_real*0.6, distance=50)
#     plt.plot(pks_real, Angle_real[pks_real], 'rx', ms=14)

    pks_virt, _ = find_peaks(Angle_virt, height=meanMaxH_virt*0.6, distance=50)
#     plt.plot(pks_virt, Angle_virt[pks_virt], 'rx', ms=14)

    plt.plot(Angle_real,'r')
    plt.plot(Angle_virt,'g')
    plt.plot(pks_real, Angle_real[pks_real], 'rx', ms=14)
    plt.plot(pks_virt, Angle_virt[pks_virt], 'rx', ms=14)

    # Lag based on difference between angular peaks 
    sampleTime = 1000 * ( np.round( (1.0 / (len(times) / trialDuration)), 3 ))
    lag = []
    for c, p in enumerate(pks_real):
        try:
            if p-4 <= pks_virt[c] <= p+4:
                lag.append(sampleTime * (pks_virt[c] - p))
        except Exception as e:
            print('Err: ', e)

    averageLag = np.round(np.nanmean(lag),3)
    #     print('Lag: ', int(np.round(averageLag)))
    
    return averageLag

In [ ]:
def ComputeErrors(df_in):
    masktemp = (df_in['gameObjectName'] == 'realFingerTip')
    adaptationTrialNumbers = pd.unique(df_in[masktemp].trialNumber)
    # np.random.shuffle(adaptationTrialNumbers)

    df_out = None # If arrays are to be saved use this 
    dat_List = []

    for i in range(len(adaptationTrialNumbers)):
        realFingerMask = (df_in['gameObjectName'] == 'realFingerTip') & (df_in['trialNumber'] == adaptationTrialNumbers[i])
        virtualFingerMask = (df_in['gameObjectName'] == 'r_index_fingernail_marker') & (df_in['trialNumber'] == adaptationTrialNumbers[i])
        ptxMask = (df_in['gameObjectName'] == 'r_index_fingernail_marker') & (df_in['trialNumber'] == adaptationTrialNumbers[i])

        # timeMask = df.loc[(df['gameObjectName'] == 'realFingerTip') & (df['trialNumber'] == adaptationTrialNumbers[i]), ['time']]
        timeMask = df_in['trialNumber'] == adaptationTrialNumbers[i]
        
        try:
            ptx = df_in[ptxMask]['PtxID'].values[0] # This only results in one participant from the cohort 

            plt.figure(1) # All positions
            plt.plot(df_in[realFingerMask].xPos, df_in[realFingerMask].zPos,'r')
            plt.plot(df_in[virtualFingerMask].xPos, df_in[virtualFingerMask].zPos,'g')
            plt.title('X-Z Position / m')
            plt.gca().set_aspect('equal', adjustable='box')
            plt.gca().set_aspect('equal', adjustable='box')
            plt.legend(['Real','Virtual'])

            ax = plt.figure(2)
            tangXZ_Real = np.sqrt(np.power(df_in[realFingerMask].xPos,2) + np.power(df_in[realFingerMask].zPos,2))
            tangXZ_Virt = np.sqrt(np.power(df_in[virtualFingerMask].xPos,2) + np.power(df_in[virtualFingerMask].zPos,2))

            print('Trial: ', i)

            try:
                tangXZ_Real_Vel = np.abs(np.diff(savgol_filter(tangXZ_Real, 75, 4)))
                tangXZ_Virt_Vel = np.abs(np.diff(savgol_filter(tangXZ_Virt, 75, 4)))


                print('Past issue: ', i)

                plt.plot(tangXZ_Real_Vel,'r')
                plt.plot(tangXZ_Virt_Vel,'g')
                plt.title('Lateral Velocity (x-z axis) $\mathregular{ms^{-1}}$')

                # Sampling Frequency and Time
                times = df_in[timeMask].time.tolist()
                val = np.where(times == numpy.amin(times))
                startTimeIdx = val[0][0]
                # print('min: ', startTimeIdx)

                print('Past 2nd issue: ', i)

                plt.figure(3) 
                plt.plot(times)
                plt.plot(startTimeIdx, times[startTimeIdx], 'rx')
                # print('Movement duration: ', times[-1], 's')
                startMovIdx = int(np.round((startTimeIdx/10))) # Convert between time series and movement array by dividing by 10? 
                sampleFreq = np.round(len(tangXZ_Real_Vel[startMovIdx:])/times[-1])
                # print('Sampling Freq: ', sampleFreq)

                print('Past 3rd issue: ', i)

                #--------------- Cross-Correlation ---------------------------------------------
                corr = np.correlate(tangXZ_Real_Vel - np.mean(tangXZ_Real_Vel), 
                                  tangXZ_Virt_Vel - np.mean(tangXZ_Virt_Vel),
                                  mode='full')
                sampleDifference = np.argmax(tangXZ_Virt_Vel[20:]) - np.argmax(tangXZ_Real_Vel[20:])

                print('Past correlation issue: ', i)

                if sampleDifference > 50:
                    lag = (sampleDifference  * (1/sampleFreq)) * 1000
                    print('Lag is too large: ' , lag)
                else: 
                    lag = (sampleDifference  * (1/sampleFreq)) * 1000
                    # print('Lag: ', np.round(lag), 'ms')

                    #--------------- Positional-Error ---------------------------------------------
                    # MSE = np.sum(np.power(np.abs((tangXZ_Real.values[20:] - tangXZ_Virt.values[20:]),2))) / len(tangXZ_Real.values[20:])

                    print('Past if statement: ', i)

                    MSE = np.round(np.sum(np.power(np.abs(tangXZ_Real.values[20:] - tangXZ_Virt.values[20:]),2)) / len(tangXZ_Real.values[20:]),4) * 100 # Convert to cm
                    endPosError = (np.round(np.nanmean(np.abs(tangXZ_Virt.values[-1-30:-1] - tangXZ_Real.values[-1-30:-1])),3)/30) * 100 # Convert to cm
                    posError = np.round(np.nanmean(np.abs(tangXZ_Virt.values[20:] - tangXZ_Real.values[20:])),3) * 100 # Convert to cm
                    velError = np.round(np.nanmean(np.abs(tangXZ_Virt_Vel[20:] - tangXZ_Real_Vel[20:])),3) # Convert to cm

                    # print('Mean Square Error: ', MSE, 'cm')
                    # print('Target Hit Error: ', endPosError, 'cm')
                    # print('Average Positional Error: ', posError, 'cm')
                    # print('Velocity Error: ', velError, 'ms-1')

                    dat_List.append([lag, MSE, endPosError, posError, velError, times[-1]])



            except Exception as e:
                print('MY_ERROR: Size of array: ', len(tangXZ_Real))
                zeroArray = np.tile(np.zeros, (1, 100))
                tangXZ_Real_Vel = zeroArray
                tangXZ_Virt_Vel = zeroArray
                
        except Exception as ex:
            print('My_Error_2: ', ex)
    

    df_out = pd.DataFrame(dat_List, columns =['Lag' ,'MSE_Error', 'Hit_Error', 'AvPos_Error', 'Vel_Error', 'Time'])
    # times = np.arange(0,)
    df_out.insert(0, 'PtxID', ptx, True)

    return df_out 

In [ ]:
def ReadAllData(path, ptxID):
    
    df = None
    
    # Get all files in the folder
    files = os.listdir(path) 
    print('Num of files: ', len(files))
    
    # Load each file into the data frame 
    for i in range(len(files)): 
        
#         print('File name: ', files[i])
#         try:
        # Extract file name info and add to the dataframe 
        fileWords = files[i].split("_") 

        tidx = fileWords.index("Trial")
        trialNum = fileWords[tidx + 1]

        # Extract user ID 
        # idx = fileWords.index("163*")
        userID = fileWords[0]


        #------------ Add to data frame ----------------
        path_file = path + "/" + files[i]
        tmpDF = pd.read_json(path_file)

        # Add user ID to data frame 
        tmpDF.insert(0, "UserID", userID, True)

        # Add Trial Number to data frame 
        tmpDF.insert(0, "TrialNum", trialNum, True)
        
        # Figure out target panel height mathematically before assigning it
        height_val = np.nanmean(tmpDF['yTPos'].values)
        
        # Extract height 
        if height_val < 0.4:
            height = "low"
        elif height_val < 0.5:
            height = "mid"
        elif height_val < 0.7:
            height = "high"
        else:
            height = "noHeight"
            
#         # Compute actual sampling rate
#         timetaken = tmpDF['time'].values
#         indexOfStart = np.where(timetaken == 0.0)
        
        # Compute actual sampling rate
        timetaken = tmpDF['time'].values
#         timetaken2 = ResizeArray(timetaken, newArrLength)
        timetaken3 = np.round(timetaken,1)
        timetaken3 = timetaken3.tolist()

        # If 0.0 time isn't present, then use the smallest time value as the start of the trial
        try:
            indexOfStart = timetaken3.index(0.0) # indexOfStart = np.where(timetaken == 0.0)
        except Exception as e:
            print('IndexErr: ', e)
            minTimeVal = np.nanmin(timetaken3)
            indexOfStart = timetaken3.index(minTimeVal)
        
        # If 0.0 time isn't present, then use the smallest time value as the start of the trial
#         samplingRate = np.round(1.0 / ((len(timetaken)-indexOfStart[0][0]) / timetaken[-1]), 4)
        try:
            samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / timetaken3[-1]), 4)
        except Exception as e:
            print('SampleErr: ', e)
            lastMaxTimeVal = np.nanmax(timetaken3[-1-10:-1])
            samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / lastMaxTimeVal), 4)
        
        
        # Get individual velocities 
        pos_x = tmpDF['xPos'].values
        pos_xf = savgol_filter(pos_x, 21, 9)
        vel_x = np.gradient(pos_xf / samplingRate)
        pos_y = tmpDF['yPos'].values
        pos_yf = savgol_filter(pos_y, 21, 9)
        vel_y = np.gradient(pos_yf / samplingRate)
        pos_z = tmpDF['zPos'].values
        pos_zf = savgol_filter(pos_z, 21, 9)
        vel_z = np.gradient(pos_zf / samplingRate)
        vel_type_1 = np.sqrt(np.power(vel_x,2) + np.power(vel_y,2) + np.power(vel_z,2))
        vel_type_1f = savgol_filter(vel_type_1, 21, 9)

        # Extract tempot 
        if "80" in fileWords or "slow" in fileWords:
            tempo = "80"
        elif "120" in fileWords or "medium" in fileWords:
            tempo = "120"
        elif "160" in fileWords or "fast" in fileWords:
            tempo = "160"
        else: 
            tempo = "noTempo"
                
        # Add height to data frame
        tmpDF.insert(0, "height", height, True)

        # Add tempo to data frame 
        tmpDF.insert(0, "tempos", tempo, True)        
        
        if df is None:
            df = tmpDF
        else:
            df = pd.concat((df, tmpDF))

#         except Exception as e:
#             print('My err: ', e)

    return df

In [ ]:
def ReadAllDataBend(path):
    # # Define data frame variable
    df = None 
    df_endpoints = None
    
    folders = os.listdir(path) 
    
    for p in range(len(folders)):

        # Extract folder names info and add to the dataframe 
        folderWords = folders[p].split("_")

        if "2021" in folders[p]: # os.path.isdir(path + folders[p]) and 

            # Extract participant name 
            idx = folderWords.index("2021")
            ptxID = folderWords[idx - 3]
            print(ptxID)
#             print(folders[p])

            # Get all files in the folder
            files = os.listdir(path + folders[p]) 

#             print(p, ' 1: Inside first loop...')

            # Load each file into the data frame 
            for i in range(len(files)): 

#                 print('2: Inside second loop...')

                # if ".json" in files[i] and height in files[i] and tempo in files[i]:  

                try:
                    # Extract file name info and add to the dataframe 
                    fileWords = files[i].split("_") 
#                     print (fileWords)
                    
                    tidx = fileWords.index("Trial")
                    trialNum = fileWords[tidx + 1]
                    
                    # Extract height 
                    if "low" in fileWords:
                        height = "low"
                    elif "mid" in fileWords:
                        height = "mid"
                    elif "high" in fileWords:
                        height = "high"
                    else:
                        height = "noHeight"

                    # Extract tempot 
                    if "80" in fileWords or "slow" in fileWords:
                        tempo = "80"
                    elif "120" in fileWords or "medium" in fileWords:
                        tempo = "120"
                    elif "160" in fileWords or "fast" in fileWords:
                        tempo = "160"
                    else: 
                        tempo = "noTempo"

                    # Extract user ID 
                    # idx = fileWords.index("163*")
                    userID = fileWords[0]


                    #------------ Add to data frame ----------------

                    tmpDF = pd.read_json(path + folders[p] + "/" + files[i])

                    # Add height to data frame
                    tmpDF.insert(0, "height", height, True)

                    # Add tempo to data frame 
                    tmpDF.insert(0, "tempo", tempo, True)

                    # Add user ID to data frame 
                    tmpDF.insert(0, "UserID", userID, True)

                    # Add Trial Number to data frame 
                    tmpDF.insert(0, "TrialNum", trialNum, True)
                    
                    # Add participant ID to data frame 
                    tmpDF.insert(0, "PtxID", ptxID, True)

                    if df is None:
                        df = tmpDF
                    else:
                        df = pd.concat((df, tmpDF))
#                         print('3: At the end of the loop...')
                    
    #-------------------------------------------------------------------------------------------------
    
#                     # Virtual finger tip terminal position
#                 end_points_x = df_all[mask_2_virt & (df_all['gameObjectName'] == 'r_index_fingernail_marker')]['xPos'].values[-1-10:-1]
#                 end_points_z = df_all[mask_2_virt & (df_all['gameObjectName'] == 'r_index_fingernail_marker')]['zPos'].values[-1-10:-1]
#                 vRow_A_point_x.append(np.round(end_points_x,5))
#                 vste_x.append(np.round(np.nanstd(end_points_x) / np.sqrt(len(end_points_x)), 5))
#                 vRow_A_point_z.append(np.round(end_points_z,5))
#                 vste_z.append(np.round(np.nanstd(end_points_z) / np.sqrt(len(end_points_z)), 5))
                
#                     Row_A_point_x = np.nanmean(tmpDF[(df_all['gameObjectName'] == 'realFingerTip')]['xPos'].values[-1-30:-1], axis=0)
#                     Row_A_point_z = np.nanmean(tmpDF[(df_all['gameObjectName'] == 'realFingerTip')]['zPos'].values[-1-30:-1], axis=0)
#                     vRow_A_point_x = np.nanmean(tmpDF[(df_all['gameObjectName'] == 'r_index_fingernail_marker')]['xPos'].values[-1-30:-1], axis=0)
#                     vRow_A_point_z = np.nanmean(tmpDF[(df_all['gameObjectName'] == 'r_index_fingernail_marker')]['zPos'].values[-1-30:-1], axis=0)

#                     ste_x.append(np.round(np.nanstd(end_points_x) / np.sqrt(len(end_points_x)), 5))

                    
#                     dataEndP = zip(Row_A_point_x, Row_A_point_z, ste_x, ste_z, vRow_A_point_x , vRow_A_point_z, vste_x, vste_z, targets)
#                     tmpDF = pd.DataFrame(dataEndP, columns=['Real_X_Pos','Real_Z_Pos', 'Real_SE_X',  'Real_SE_Z', 'Virtual_X_Pos','Virtual_Z_Pos', 'Virtual_SE_X',  'Virtual_SE_Z', 'targetID'])    

#                     tmpDF_endpoint = 
        
#                     if df_endpoints is None:
#                         df_endpoints = tmpDF_endpoint
#                     else:
#                         df_endpoints = pd.concat((df_endpoints, tmpDF_endpoint))
    
    
                except Exception as e:
                    print('My err: ', e)

    return df

In [ ]:
def CrossCorr2(vel_1, vel_2, sampleFreq):
    import operator
    
    # Detrend data first: 
    response = mlb.detrend(vel_1)
    signal = mlb.detrend(vel_2)
    max_corr = np.zeros(6)
    max_lag = np.zeros(6)
    
    # Compute lag at max lag
    try:
        lags,c,line,b = plt.xcorr(response, signal,usevlines = False,normed=True)
        
#         plt.figure()
#         plt.subplot(1,1,1)
#         corr_6 = plt.plot(lags,c,color='g')

        #         c = np.sqrt(c*c)
        index, value = max(enumerate(c), key=operator.itemgetter(1))
        max_corr[5] = value
        max_lag[5] = lags[index]
    except Exception as e:
        print(e)
        max_corr[5] = np.nan
        max_lag[5] = np.nan
        
    return max_lag[5]

In [ ]:
def SamplingRate(timeValues):
    # Compute actual sampling rate
    timetaken = timeValues
#     timetaken2 = ResizeArray(timetaken, newArrLength)
    timetaken3 = np.round(timetaken,1)
    timetaken3 = timetaken3.tolist()

    # If 0.0 time isn't present, then use the smallest time value as the start of the trial
    try:
        indexOfStart = timetaken3.index(0.0) # indexOfStart = np.where(timetaken == 0.0)
    except Exception as e:
        print('IndexErr: ', e)
        minTimeVal = np.nanmin(timetaken3)
        indexOfStart = timetaken3.index(minTimeVal)

    # Check if devision by zero, because the last value happens to be zero and use last largest value instead
    try:
        samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / timetaken3[-1]), 4)
    except Exception as e:
        print('SampleErr: ', e)
        lastMaxTimeVal = np.nanmax(timetaken3[-1-10:-1])
        samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / lastMaxTimeVal), 4)

#     print('Sampling Rate: ', np.round(1.0/samplingRate), ' Hz')
    
    return np.round(1.0/samplingRate)

# New Section

Load all the files into a single data frame. This should be updated to create 5 separate data frames: (1) Base line, (2) Adaptation, (3) Washout, (4) Catch and (5) Questions 

# Real all files

In [ ]:
pathBend = 'E:/Projects/QuestAccuracyAnalysis/Quest Accuracy Data-20211103T165650Z-001/Quest Accuracy Data/AAA_Diar_Strange/benddata/'
pathBend

In [ ]:
folders = os.listdir(pathBend)
print(len(folders))
print(folders)

startTime = time.time()

df_allBend = ReadAllDataBend(pathBend)
# df_all = pd.read_json(path + "latest_df_all.json")

endTime = time.time()
computeDuration = endTime - startTime
print('time: ', computeDuration)

In [ ]:
print(pd.unique(df_allBend['gameObjectName']))

In [ ]:
realMask1 = (df_allBend['UserID'] == '1634123474') & (df_allBend['gameObjectName'] == 'realFingerTip')
realMask2 = (df_allBend['UserID'] == '1634123474') & (df_allBend['gameObjectName'] == 'realFingerTip (1)')
realMask3 = (df_allBend['UserID'] == '1634123474') & (df_allBend['gameObjectName'] == 'realFingerTip (2)')
realMask4 = (df_allBend['UserID'] == '1634123474') & (df_allBend['gameObjectName'] == 'realFingerTip (3)')

virtMask1 = (df_allBend['UserID'] == '1634123474') & (df_allBend['gameObjectName'] == 'r_index_fingernail_marker')
virtMask2 = (df_allBend['UserID'] == '1634123474') & (df_allBend['gameObjectName'] == 'b_r_index1')
virtMask3 = (df_allBend['UserID'] == '1634123474') & (df_allBend['gameObjectName'] == 'b_r_index2')
virtMask4 = (df_allBend['UserID'] == '1634123474') & (df_allBend['gameObjectName'] == 'b_r_index3')

timeVals = df_allBend[virtMask1]['time']  
sR = SamplingRate(timeVals)
interSampleInterval = 1/sR #0.01

startIdx = 450
endIdx = 500

# Marker 1
pos_x_m1 = df_allBend[realMask1]['xPos'][startIdx:endIdx]
pos_y_m1 = df_allBend[realMask1]['yPos'][startIdx:endIdx]
pos_xf = savgol_filter(pos_x_m1, 21, 9)
pos_yf = savgol_filter(pos_y_m1, 21, 9)
pos_xv_m1 = df_allBend[virtMask1]['xPos'][startIdx:endIdx]
pos_yv_m1 = df_allBend[virtMask1]['yPos'][startIdx:endIdx]
pos_xvf = savgol_filter(pos_xv_m1, 21, 9)
pos_yvf = savgol_filter(pos_yv_m1, 21, 9)

# Position
plt.figure
plt.subplot(1,2,1)
plt.plot(pos_x_m1, pos_y_m1,'k-o')
plt.plot(pos_xv_m1,pos_yv_m1,'m-o')
plt.legend(['Real','Virtual'])
plt.plot([pos_x_m1, pos_xv_m1], [pos_y_m1, pos_yv_m1],'b-',alpha=0.5)
plt.ylim([0.5, 0.9])
plt.axis('equal')

# Velocity Real
vel_x = np.gradient(pos_xf / interSampleInterval)
vel_y = np.gradient(pos_yf / interSampleInterval)
vel_type_1 = np.sqrt(np.power(vel_x,2) + np.power(vel_y,2))
vel_real = savgol_filter(vel_type_1, 21, 9)

# Velocity Virtual
vel_xv = np.gradient(pos_xvf / interSampleInterval)
vel_yv = np.gradient(pos_yvf / interSampleInterval)
vel_type_1v = np.sqrt(np.power(vel_xv,2) + np.power(vel_yv,2))
vel_virt = savgol_filter(vel_type_1v, 21, 9)

plt.subplot(1,2,2)
plt.plot(vel_real,'k')
plt.plot(vel_virt,'m')
plt.ylim([0, 1.5])

lag = CrossCorr(vel_real, vel_virt, interSampleInterval)
print('Lag: ', lag, ' ms')

In [ ]:
# path = "/content/drive/MyDrive/Colab Notebooks/Quest Accuracy Data/QuestTrackingData/Pete_5th_Oct_2021/"
path = "E:/Projects/QuestAccuracyAnalysis/Quest Accuracy Data-20211103T165650Z-001/Quest Accuracy Data/QuestTrackingData/"
print(path)

# # Define data frame variable
df_all = None
df_tmp = None 

df_Max = None
df_Pet = None
df_Kat = None

folders = os.listdir(path)

for i, p in enumerate(folders):
    
    # Start timer 
    startTime = time.time()
    
    print('idx: ', i ,' Folder: ', p)
    
    # Extract folder names info and add to the dataframe 
    folderWords = folders[i].split("_")
    idx = folderWords.index("2021")
    ptxID = folderWords[idx - 3]
    print('Ptx ID: ', ptxID, '**************************************')
    
    if "Max" in ptxID: 
        df_tmmp = ReadAllData(path_folder, ptxID)     # df_all = pd.read_json(path + "latest_df_all.json")
        df_tmmp.insert(0, "PtxID", ptxID, True) # Add participant ID to data frame 
        df_tmmp = df_tmmp[['PtxID','TrialNum','UserID', 'tempos', 'height', 'frameNum', 'gameObjectName', 'xPos', 'yPos','zPos','xRot', 'yRot', 'zRot', 'targetID', 'xTPos', 'yTPos', 'zTPos', 'time']]
        df_Max = df_tmmp
#         continue
    if "Pete" in ptxID: 
        df_tmmp = ReadAllData(path_folder, ptxID)     # df_all = pd.read_json(path + "latest_df_all.json")
        df_tmmp.insert(0, "PtxID", ptxID, True) # Add participant ID to data frame 
        df_tmmp = df_tmmp[['PtxID','TrialNum','UserID', 'tempos', 'height', 'frameNum', 'gameObjectName', 'xPos', 'yPos','zPos','xRot', 'yRot', 'zRot', 'targetID', 'xTPos', 'yTPos', 'zTPos', 'time']]
        df_Pet = df_tmmp
#         continue
    if "Katrina" in ptxID: 
        df_tmmp = ReadAllData(path_folder, ptxID)     # df_all = pd.read_json(path + "latest_df_all.json")
        df_tmmp.insert(0, "PtxID", ptxID, True) # Add participant ID to data frame 
        df_tmmp = df_tmmp[['PtxID','TrialNum','UserID', 'tempos', 'height', 'frameNum', 'gameObjectName', 'xPos', 'yPos','zPos','xRot', 'yRot', 'zRot', 'targetID', 'xTPos', 'yTPos', 'zTPos', 'time']]
        df_Kat = df_tmmp
#         continue
    else:     
        try:
            path_folder = path + p + "/"
            df_tmp = ReadAllData(path_folder, ptxID)     # df_all = pd.read_json(path + "latest_df_all.json")
            df_tmp.insert(0, "PtxID", ptxID, True) # Add participant ID to data frame 

            df_tmp2 = df_tmp[['PtxID','TrialNum','UserID', 'tempos', 'height', 'frameNum', 'gameObjectName', 'xPos', 'yPos','zPos','xRot', 'yRot', 'zRot', 'targetID', 'xTPos', 'yTPos', 'zTPos', 'time']]

            if df_all is None:
                df_all = df_tmp2
            else:
                df_all = pd.concat((df_all, df_tmp2))

        except Exception as e: 
            print('Err: ', e)
            continue

    endTime = time.time()
    computeDuration = endTime - startTime
    print('Processing time: ', np.round(computeDuration/60.0), ' minutes')    

### Fix issues with Joe's and Max's data sets
##### These are data frames with duplicate columns, for some unexplainable reason. As a result the duplicate columns have to be removed first before we can add them to the main data frame 

###### This is to correct the following error: InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
df_Kat =df_Kat.loc[:,~df_Kat.columns.duplicated()]
df_Max =df_Max.loc[:,~df_Max.columns.duplicated()]
df_Pet =df_Pet.loc[:,~df_Pet.columns.duplicated()]

df_all = pd.concat((df_all, df_Kat))
df_all = pd.concat((df_all, df_Max))
df_all = pd.concat((df_all, df_Pet))

In [ ]:
pd.unique(df_all['PtxID'])

In [ ]:
ptxNum = 6
ptxIds = pd.unique(df_all['PtxID'])
print('Ptx: ', ptxIds[ptxNum])

temps = pd.unique(df_all['tempos'])
print('Tempos: ', temps)


ptx_tempo_Mask_80 = (df_all['PtxID'] == ptxIds[ptxNum]) & (df_all['tempos'] == temps[2])

tempores1 = pd.unique(df_all[ptx_tempo_Mask_80]['tempos'])
ptx_tempo_Mask_80 = (df_all['PtxID'] == ptxIds[ptxNum]) & (df_all['tempos'] == temps[1])
tempores2 = pd.unique(df_all[ptx_tempo_Mask_80]['tempos'])
ptx_tempo_Mask_80 = (df_all['PtxID'] == ptxIds[ptxNum]) & (df_all['tempos'] == temps[0])
tempores3 = pd.unique(df_all[ptx_tempo_Mask_80]['tempos'])

print(temps[2], ': ', tempores1, '\n', temps[1], ': ', tempores2,'\n', temps[0], ': ', tempores3)

## Plot data from individual participants 

In [ ]:
mask = (df_all['PtxID'] == 'Susan') & (df_all['tempos'] == '160') & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == 'low')
targs = pd.unique(df_all[mask]['targetID'])
np.sort(targs)

In [ ]:
ptxIds

In [ ]:
# # print('Trial: ', 23)    
newArrLength = 100
height = 'low'
temp = '160'
targID = 'row_C4'

mask = (df_all['tempos'] == temp) & (df_all['height'] == height) & (df_all['targetID'] == targID) & (df_all['PtxID'] == ptxIds[0]) & (df_all['gameObjectName'] == 'realFingerTip') 
mask_virt = (df_all['tempos'] == temp) & (df_all['height'] == height) & (df_all['targetID'] == targID) & (df_all['PtxID'] == ptxIds[0]) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') 
VisualizeTrajectories(mask, mask_virt, newArrLength)

suspectTrials = pd.unique(df_all[mask]['TrialNum'])
print(suspectTrials)

In [ ]:
suspectTrials = pd.unique(df_all[mask]['TrialNum'])
print(suspectTrials)

In [ ]:
# Ptx:  Susan  Trial:  21  TargetID:  row_C3  tempo:  160  height:  low
%matplotlib inline
plt.rcParams["figure.figsize"] = (4,5)
sns.set_theme(style="whitegrid")

trialNumbs = '70' #'21'

mask = (df_all['TrialNum'] == trialNumbs) & (df_all['tempos'] == temp) & (df_all['height'] == height) & (df_all['targetID'] == targID) & (df_all['PtxID'] == ptxIds[0]) & (df_all['gameObjectName'] == 'realFingerTip') 
mask_virt = (df_all['TrialNum'] == trialNumbs) & (df_all['tempos'] == temp) & (df_all['height'] == height) & (df_all['targetID'] == targID) & (df_all['PtxID'] == ptxIds[0]) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') 

saveFig = True
VisualizeTrajectories(mask, mask_virt, newArrLength, saveFig) 

In [ ]:
# realMask1 = (df_all['UserID'] == '1634123474') & (df_all['gameObjectName'] == 'realFingerTip')
# realMask2 = (df_all['UserID'] == '1634123474') & (df_all['gameObjectName'] == 'realFingerTip (1)')
# realMask3 = (df_all['UserID'] == '1634123474') & (df_all['gameObjectName'] == 'realFingerTip (2)')
# realMask4 = (df_all['UserID'] == '1634123474') & (df_all['gameObjectName'] == 'realFingerTip (3)')

# virtMask1 = (df_all['UserID'] == '1634123474') & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
# virtMask2 = (df_all['UserID'] == '1634123474') & (df_all['gameObjectName'] == 'b_r_index1')
# virtMask3 = (df_all['UserID'] == '1634123474') & (df_all['gameObjectName'] == 'b_r_index2')
# virtMask4 = (df_all['UserID'] == '1634123474') & (df_all['gameObjectName'] == 'b_r_index3')

# timeVals = df_all[virtMask1]['time']  
# sR = SamplingRate(timeVals)
# interSampleInterval = 1/sR #0.01

# startIdx = 450
# endIdx = 500

# # Marker 1
# pos_x_m1 = df_all[realMask1]['xPos'][startIdx:endIdx]
# pos_y_m1 = df_all[realMask1]['yPos'][startIdx:endIdx]
# pos_xf = savgol_filter(pos_x_m1, 21, 9)
# pos_yf = savgol_filter(pos_y_m1, 21, 9)
# pos_xv_m1 = df_all[virtMask1]['xPos'][startIdx:endIdx]
# pos_yv_m1 = df_all[virtMask1]['yPos'][startIdx:endIdx]
# pos_xvf = savgol_filter(pos_xv_m1, 21, 9)
# pos_yvf = savgol_filter(pos_yv_m1, 21, 9)

# # Position
# plt.figure
# plt.subplot(1,2,1)
# plt.plot(pos_x_m1, pos_y_m1,'k-o')
# plt.plot(pos_xv_m1,pos_yv_m1,'m-o')
# plt.legend(['Real','Virtual'])
# plt.plot([pos_x_m1, pos_xv_m1], [pos_y_m1, pos_yv_m1],'b-',alpha=0.5)
# plt.ylim([0.5, 0.9])
# plt.axis('equal')

# # Velocity Real
# vel_x = np.gradient(pos_xf / interSampleInterval)
# vel_y = np.gradient(pos_yf / interSampleInterval)
# vel_type_1 = np.sqrt(np.power(vel_x,2) + np.power(vel_y,2))
# vel_real = savgol_filter(vel_type_1, 21, 9)

# # Velocity Virtual
# vel_xv = np.gradient(pos_xvf / interSampleInterval)
# vel_yv = np.gradient(pos_yvf / interSampleInterval)
# vel_type_1v = np.sqrt(np.power(vel_xv,2) + np.power(vel_yv,2))
# vel_virt = savgol_filter(vel_type_1v, 21, 9)

# plt.subplot(1,2,2)
# plt.plot(vel_real,'k')
# plt.plot(vel_virt,'m')
# plt.ylim([0, 1.5])

# lag = CrossCorr(vel_real, vel_virt, interSampleInterval)
# print('Lag: ', lag, ' ms')

In [ ]:
trialNumArr = np.arange(0,72)
height = 'mid'
temp = '160'

for trialNum in trialNumArr:
    
#     print('Participant: ', ptxIds[0], 'Trial: ', trialNum)
#     print('Participant: ', ptxIds[0], 'Trial: ', trialNum)
    try:
        print('Trial: ', trialNum)    
        mask = (df_all['targetID'] == 'row_C6') & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == height)
        mask_virt = (df_all['targetID'] == 'row_C6') & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['height'] == height)
        VisualizeTrajectories(mask, mask_virt)
    except Exception as e:
        print(e)
        
#     mask = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[0]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == height)
#     mask_virt = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[0]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['height'] == height)
#     VisualizeTrajectories(mask, mask_virt)

#     print('Participant: ', ptxIds[1], 'Trial: ', trialNum)
#     mask = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[1]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == height)
#     mask_virt = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[0]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['height'] == height)
#     VisualizeTrajectories(mask, mask_virt)

#     print('Participant: ', ptxIds[2], 'Trial: ', trialNum)
#     mask = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[2]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == height)
#     mask_virt = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[0]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['height'] == height)
#     VisualizeTrajectories(mask, mask_virt)
    
#     print('Participant: ', ptxIds[3], 'Trial: ', trialNum)
#     mask = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[3]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == height)
#     mask_virt = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[0]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['height'] == height)
#     VisualizeTrajectories(mask, mask_virt)

#     print('Participant: ', ptxIds[4], 'Trial: ', trialNum)
#     mask = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[4]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == height)
#     mask_virt = (df_all['TrialNum'] == str(trialNum)) & (df_all['PtxID'] == ptxIds[0]) & (df_all['tempos'] == temp) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['height'] == height)
#     VisualizeTrajectories(mask, mask_virt)

In [ ]:
df_metrics_80_low = ExtractMetrics(df_all, '80', 'low')

In [ ]:
df_metrics_120_low = ExtractMetrics(df_all, '120', 'low')

In [ ]:
df_metrics_160_low = ExtractMetrics(df_all, '160', 'low')

In [ ]:
df_metrics_80_mid = ExtractMetrics(df_all, '80', 'mid')

In [ ]:
df_metrics_120_mid = ExtractMetrics(df_all, '120', 'mid')

In [ ]:
df_metrics_160_mid = ExtractMetrics(df_all, '160', 'mid')

In [ ]:
df_metrics_80_high = ExtractMetrics(df_all, '80', 'high')

In [ ]:
df_metrics_120_high = ExtractMetrics(df_all, '120', 'high')

In [ ]:
df_metrics_160_high = ExtractMetrics(df_all, '160', 'high')

In [ ]:
#pd.unique(df_metrics_80_low['TargetID'])
#pd.unique(df_metrics_120_low['TargetID']) # Got extra weird targets
#pd.unique(df_metrics_160_low['TargetID'])

#pd.unique(df_metrics_80_mid['TargetID'])
#pd.unique(df_metrics_120_mid['TargetID']) # Got extra weird targets
#pd.unique(df_metrics_160_mid['TargetID'])

#pd.unique(df_metrics_80_high['TargetID'])
#pd.unique(df_metrics_120_high['TargetID'])
#pd.unique(df_metrics_160_high['TargetID'])

In [ ]:
df_metrics_80_low.insert(2, 'tempos', '80')
df_metrics_80_low.insert(2, 'height', 'low')

df_metrics_120_low.insert(2, 'tempos', '120')
df_metrics_120_low.insert(2, 'height', 'low')

df_metrics_160_low.insert(2, 'tempos', '160')
df_metrics_160_low.insert(2, 'height', 'low')

df_metrics_80_mid.insert(2, 'tempos', '80')
df_metrics_80_mid.insert(2, 'height', 'mid')

df_metrics_120_mid.insert(2, 'tempos', '120')
df_metrics_120_mid.insert(2, 'height', 'mid')

df_metrics_160_mid.insert(2, 'tempos', '160')
df_metrics_160_mid.insert(2, 'height', 'mid')

df_metrics_80_high.insert(2, 'tempos', '80')
df_metrics_80_high.insert(2, 'height', 'high')

df_metrics_120_high.insert(2, 'tempos', '120')
df_metrics_120_high.insert(2, 'height', 'high')

df_metrics_160_high.insert(2, 'tempos', '160')
df_metrics_160_high.insert(2, 'height', 'high')

In [ ]:
df_metrics_all = df_metrics_80_low
df_metrics_all = pd.concat((df_metrics_all, df_metrics_120_low))
df_metrics_all = pd.concat((df_metrics_all, df_metrics_160_low))

df_metrics_all = pd.concat((df_metrics_all, df_metrics_80_mid))
df_metrics_all = pd.concat((df_metrics_all, df_metrics_120_mid))
df_metrics_all = pd.concat((df_metrics_all, df_metrics_160_mid))

df_metrics_all = pd.concat((df_metrics_all, df_metrics_80_high))
df_metrics_all = pd.concat((df_metrics_all, df_metrics_120_high))
df_metrics_all = pd.concat((df_metrics_all, df_metrics_160_high))

df_metrics_all

In [ ]:
# Save data frames to file

df_metrics_all.to_csv('df_metrics_all.csv', index=False)
compression_opts = dict(method='zip', archive_name='df_all.csv')  
df_all.to_csv('df_all.zip', index=False, compression=compression_opts)

# Start from here by loading previously saved dataframes

In [ ]:
df_metrics_all = pd.read_csv (r'df_metrics_all.csv')
df_metrics_all

In [ ]:
df_all = pd.read_csv('df_all.zip', compression='zip')
df_all

### Sort out weird targets in the df_metrics_all data frame

In [ ]:
weirdTargetIDs = ['0    row_C4\n0    row_B2\nName: targetID, dtype: object',
                  '0    row_B3\n0    row_A3\nName: targetID, dtype: object',
                  '0    row_C6\n0    row_A2\nName: targetID, dtype: object',
                  '0    row_C2\n0    row_C2\nName: targetID, dtype: object',
                  '0    row_B5\n0    row_C1\nName: targetID, dtype: object',
                  '0    row_B2\n0    row_B5\nName: targetID, dtype: object',
                  '0    row_C3\n0    row_C6\nName: targetID, dtype: object',
                  '0    row_B2\n0    row_C3\nName: targetID, dtype: object',
                  '0    row_A1\n0    row_C2\nName: targetID, dtype: object',
                  '0    row_C4\n0    row_C4\nName: targetID, dtype: object',
                  '0    row_C1\n0    row_A2\nName: targetID, dtype: object',
                  '0    row_B3\n0    row_A4\nName: targetID, dtype: object',
                  '0    row_A3\n0    row_A6\nName: targetID, dtype: object',
                  '0    row_C5\n0    row_B4\nName: targetID, dtype: object',
                  '0    row_C3\n0    row_A1\nName: targetID, dtype: object',
                  '0    row_A6\n0    row_B5\nName: targetID, dtype: object',
                  '0    row_B5\n0    row_B1\nName: targetID, dtype: object',
                  '0    row_A2\n0    row_B6\nName: targetID, dtype: object',
                  '0    row_B4\n0    row_C1\nName: targetID, dtype: object',
                  '0    row_B6\n0    row_A5\nName: targetID, dtype: object',
                  '0    row_B1\n0    row_C5\nName: targetID, dtype: object',
                  '0    row_A5\n0    row_A3\nName: targetID, dtype: object',
                  '0    row_C6\n0    row_C6\nName: targetID, dtype: object',
                  '0    row_C2\n0    row_B3\nName: targetID, dtype: object',
                  '0    row_B2\n0    row_A4\nName: targetID, dtype: object',
                  '0    row_C3\n0    row_B2\nName: targetID, dtype: object',
                  '0    row_C2\n0    row_A1\nName: targetID, dtype: object',
                  '0    row_A2\n0    row_C1\nName: targetID, dtype: object',
                  '0    row_A4\n0    row_B3\nName: targetID, dtype: object',
                  '0    row_A6\n0    row_A3\nName: targetID, dtype: object',
                  '0    row_B4\n0    row_C5\nName: targetID, dtype: object',
                  '0    row_A1\n0    row_C3\nName: targetID, dtype: object',
                  '0    row_B5\n0    row_A6\nName: targetID, dtype: object',
                  '0    row_B1\n0    row_B5\nName: targetID, dtype: object',
                  '0    row_B6\n0    row_A2\nName: targetID, dtype: object',
                  '0    row_C1\n0    row_B4\nName: targetID, dtype: object',
                  '0    row_A5\n0    row_B6\nName: targetID, dtype: object',
                  '0    row_C5\n0    row_B1\nName: targetID, dtype: object',
                  '0    row_A3\n0    row_A5\nName: targetID, dtype: object',
                  '0    row_B3\n0    row_C2\nName: targetID, dtype: object']


print('Length of weird TargetIDs: ', len(weirdTargetIDs))
for i in weirdTargetIDs:
    mask = (df_metrics_all['TargetID'] == i)
    df_metrics_all.drop(df_metrics_all[mask].index, inplace=True)

df_metrics_all

In [ ]:
#pd.unique(df_metrics_all['TargetID'])
pd.unique(df_metrics_all['TargetID'])

### Clean up data
- [ ] Outlier detection and removal 
- [ ] using 3x std
- [ ] using interquartile
- [ ] map to normal distribution 

In [ ]:
metric1 = 'Lag'
lag_vals = df_metrics_all[df_metrics_all[metric1] < 200.0]

print('Mean Lag: ', lag_vals[metric1].mean())
print('SD Lag: ', lag_vals[metric1].std())
print('Max Lag: ', lag_vals[metric1].max())
print('Min Lag: ', lag_vals[metric1].min())

In [ ]:
# metric1 = 'Lag'
# # Remove outliers and map to normal distribution 
# data_norm = CleanUpnNorm(df_metrics_all, metric1, 1.0)
# # Re-add the corrected metrics to the dataframe
# df_metrics_all[metric1 + '_cleaned'] = data_norm
# plt.ylabel(metric1)

metric2 = 'EndError'
# Remove outliers and map to normal distribution 
data_norm = CleanUpnNorm(df_metrics_all, metric2, 0.1)
# Re-add the corrected metrics to the dataframe
df_metrics_all[metric2 + '_cleaned'] = data_norm
plt.ylabel(metric2)

metric3 = 'PathOffset'
# Remove outliers and map to normal distribution 
data_norm = CleanUpnNorm(df_metrics_all, metric3, 0.1)
# Re-add the corrected metrics to the dataframe
df_metrics_all[metric3 + '_cleaned'] = data_norm
plt.ylabel(metric3)

metric4 = 'PathOffsetNoLag'
# Remove outliers and map to normal distribution 
data_norm = CleanUpnNorm(df_metrics_all, metric4, 0.75)
# Re-add the corrected metrics to the dataframe
df_metrics_all[metric4 + '_cleaned'] = data_norm
plt.ylabel(metric4)

### Sphericity Test

In [ ]:
spher, _, chisq, dof, pval = pg.sphericity(df_metrics_all, dv='PathOffset_cleaned',
                                           subject='PtxID',
                                           within=['tempos'])
print('Spherical: ', spher, '\nChiSq: ', round(chisq, 3), '\nDof:', dof, '\np-val: ', round(pval, 3))


In [ ]:
# Tempo and height 
sns.set()

# from statsmodels.graphics.factorplots import interaction_plot
# fig = interaction_plot(df_metrics_all["tempos"], df_metrics_all["height"], df_metrics_all["EndError_cleaned"], 
#                        colors=['red','green', "blue"], markers=['D','^', 'o'], ylabel='Positional Error (cm)', xlabel='Tempo')

# sns.pointplot(data=data, x='height', y='EndError_cleaned',hue='TargetLateral', dodge=True,
#               capsize=.1, errwidth=1, palette='colorblind')

plt.figure()
fig1 = sns.pointplot(x="tempos", y="EndError_cleaned", hue="height",dodge=True, data=df_metrics_all,
                     capsize=.1, errwidth=1, palette='colorblind')
plt.figure()
fig2 = sns.pointplot(x="tempos", y="PathOffsetNoLag_cleaned", hue="height",dodge=True, data=df_metrics_all,
                     capsize=.1, errwidth=1, palette='colorblind')

plt.show()

In [ ]:
plt.figure()
fig1 = sns.pointplot(x="TargetID", y="tempos", hue="height",dodge=True, data=df_metrics_all,
                     capsize=.1, errwidth=1, palette='colorblind')

plt.show()

In [ ]:
# Target ID
plt.figure()
fig1 = sns.pointplot(x="TargetID", y="EndError_cleaned", hue="height", data=df_metrics_all, palette='Reds')

plt.figure()
fig2 = sns.pointplot(x="TargetID", y="PathOffsetNoLag_cleaned", hue="height", data=df_metrics_all, palette='Greens')

plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = (4,4)
plt.tight_layout(pad=5.0)
titleFontSize = 16

# ------------------------------------------------------------------------------
plt.figure()
ax1 = sns.barplot(
    data=df_metrics_all,
    x='tempos', 
    y='PathOffsetNoLag_cleaned',
    hue='height',
    palette = 'Blues')

# plt.title('Angular Error \n Between Quest and MoCap \n \n \n \n \n \n \n \n')
plt.ylabel('Offset (cm)')
plt.xlabel('Tempos')
plt.xticks([0,1,2],['80','120','160'])

statannot.add_stat_annotation(
    ax1,
    data=df_metrics_all,
    x='tempos',
    y='PathOffsetNoLag_cleaned',
    box_pairs=[(80, 120), (80,160), (120, 160)],
    test="t-test_ind",
    text_format="star",
    loc="outside",
)

# plt.rcParams["figure.figsize"] = (3,3)
plt.title('Pathoffset Across Tempos \n\n\n\n',fontsize=titleFontSize)
plt.savefig(str(np.round(time.time())) + '_Pathoffset Across Tempos.png', dpi=600, bbox_inches='tight')

# ------------------------------------------------------------------------------
plt.figure()
ax1 = sns.barplot(
    data=df_metrics_all,
    x='height', 
    y='PathOffsetNoLag_cleaned',
    hue='tempos',
    palette = 'Blues')

# plt.title('Angular Error \n Between Quest and MoCap \n \n \n \n \n \n \n \n')
plt.ylabel('Offset (cm)')
plt.xlabel('Heights')
plt.xticks([0,1,2],['Low','Mid','High'])

statannot.add_stat_annotation(
    ax1,
    data=df_metrics_all,
    x='height',
    y='PathOffsetNoLag_cleaned',
    box_pairs=[('low', 'mid'), ('low','high'), ('mid', 'high')],
    test="t-test_ind",
    text_format="star",
    loc="outside",
)

# plt.rcParams["figure.figsize"] = (3,3)
plt.title('Pathoffset Across Heights \n\n\n\n',fontsize=titleFontSize)
plt.savefig(str(np.round(time.time())) + '_Pathoffset Across Heights.png', dpi=600, bbox_inches='tight')

# ------------------------------------------------------------------------------
plt.figure()
ax2 = sns.barplot(
    data=df_metrics_all,
    x='TargetLateral', 
    y='PathOffsetNoLag_cleaned',
    hue='TargetVertical',
    palette = 'Blues')

# plt.title('Angular Error \n Between Quest and MoCap \n \n \n \n \n \n \n \n')
plt.ylabel('Offset (cm)')
plt.xlabel('Target Location')
plt.xticks([0,1,2],['left','center','right'])

statannot.add_stat_annotation(
    ax2,
    data=df_metrics_all,
    x='TargetLateral',
    y='PathOffsetNoLag_cleaned',
    box_pairs=[('left', 'center'), ('left','right'), ('center', 'right')],
    test="t-test_ind",
    text_format="star",
    loc="outside",
)

# plt.rcParams["figure.figsize"] = (3,3)
plt.title('Pathoffset Across Target Locations\n\n\n\n',fontsize=titleFontSize)
plt.savefig(str(np.round(time.time())) + '_Pathoffset Across Target Locations.png', dpi=600, bbox_inches='tight')

# ------------------------------------------------------------------------------
plt.figure()
ax3 = sns.barplot(
    data=df_metrics_all,
    x='TargetLateral', 
    y='EndError_cleaned',
    hue='TargetVertical',
    palette = 'Blues')

# plt.title('Angular Error \n Between Quest and MoCap \n \n \n \n \n \n \n \n')
plt.ylabel('Error (cm)')
plt.xlabel('Target Location')
plt.xticks([0,1,2],['left','center','right'])

statannot.add_stat_annotation(
    ax3,
    data=df_metrics_all,
    x='TargetLateral',
    y='EndError_cleaned',
    box_pairs=[('left', 'center'), ('left','right'), ('center', 'right')],
    test="t-test_ind",
    text_format="star",
    loc="outside",
)

# plt.rcParams["figure.figsize"] = (3,3)
plt.title('Positional Error Across Target Location \n\n\n\n',fontsize=titleFontSize)
plt.savefig(str(np.round(time.time())) + '_Positional Error Across Target Location.png', dpi=600, bbox_inches='tight')

# ------------------------------------------------------------------------------
plt.figure()
ax4 = sns.barplot(
    data=df_metrics_all,
    x='height', 
    y='EndError_cleaned',
    hue='tempos',
    palette = 'Blues')

# plt.title('Angular Error \n Between Quest and MoCap \n \n \n \n \n \n \n \n')
plt.ylabel('Error (cm)')
plt.xlabel('Heights')
plt.xticks([0,1,2],['low','mid','high'])

# statannot.add_stat_annotation(
#     ax4,
#     data=df_metrics_all,
#     x='height',
#     y='EndError_cleaned',
#     box_pairs=[('low', 'mid'), ('low','high'), ('mid', 'high')],
#     test="t-test_ind",
#     text_format="star",
#     loc="outside",
# )

plt.title('Positional Error Across Heights \n\n\n\n',fontsize=titleFontSize)
plt.savefig(str(np.round(time.time())) + '_Positional Error Across Heights.png', dpi=600, bbox_inches='tight')

# ------------------------------------------------------------------------------
plt.figure()
ax4 = sns.barplot(
    data=df_metrics_all,
    x='tempos', 
    y='EndError_cleaned',
    hue='height',
    palette = 'Blues')

# plt.title('Angular Error \n Between Quest and MoCap \n \n \n \n \n \n \n \n')
plt.ylabel('Error (cm)')
plt.xlabel('Tempos')
plt.xticks([0,1,2],['80','120','160'])

# statannot.add_stat_annotation(
#     ax4,
#     data=df_metrics_all,
#     x='tempos',
#     y='EndError_cleaned',
#     box_pairs=[('80', '120'), ('80','160'), ('120', '160')],
#     test="t-test_ind",
#     text_format="star",
#     loc="outside",
# )

plt.title('Positional Error Across Tempos \n\n\n\n',fontsize=titleFontSize)
plt.savefig(str(np.round(time.time())) + '_Positional Error Across Tempos.png', dpi=600, bbox_inches='tight')


In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (4,4)
sns.set_theme(style="whitegrid")
plt.tight_layout(pad=5.0)
titleFontSize = 16

plt.figure()
sns.boxplot(x='TargetLateral',y='EndError_cleaned', order=["left", "center", "right"],hue='TargetVertical', data=df_metrics_all, palette='Reds')
plt.title('Positional Error Across Target Locations', fontsize=titleFontSize)
plt.xlabel('Lateral Target Location')
plt.ylabel('Error (cm)')
plt.grid(False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.savefig(str(np.round(time.time())) + '_EndError_TargetLoc.png', dpi=600, bbox_inches='tight')

plt.figure()
sns.boxplot(x='height',y='EndError_cleaned',hue='tempos', data=df_metrics_all, palette='Greens')
plt.title('Positional Error Across Heights and Tempos', fontsize=titleFontSize)
plt.xlabel('Heights')
plt.ylabel('Error (cm)')
plt.grid(False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.savefig(str(np.round(time.time())) + '_EndError_Tempos.png', dpi=600, bbox_inches='tight')

plt.figure()
sns.boxplot(x='TargetLateral',y='PathOffsetNoLag_cleaned', order=["left", "center", "right"],hue='TargetVertical', data=df_metrics_all, palette='Reds')
plt.title('Path Offset Across Target Locations', fontsize=titleFontSize)
plt.xlabel('Lateral Target Location')
plt.ylabel('Offset (cm)')
plt.grid(False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.savefig(str(np.round(time.time())) + '_PathOffset_TargetLoc.png', dpi=600, bbox_inches='tight')

plt.figure()
sns.boxplot(x='height',y='PathOffsetNoLag_cleaned',hue='tempos', data=df_metrics_all, palette='Greens')
plt.title('Path Offset Across Heights and Tempos', fontsize=titleFontSize)
plt.xlabel('Heights')
plt.ylabel('Offset (cm)')
plt.grid(False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.savefig(str(np.round(time.time())) + '_PathOffset_Heights.png', dpi=600, bbox_inches='tight')


plt.figure()
sns.boxplot(x='tempos',y='Lag',hue='height', data=df_metrics_all, palette='Blues')
plt.title('Delay Across Tempos', fontsize=titleFontSize)
plt.xlabel('Tempos')
plt.ylabel('Delay (ms)')
plt.ylim([-10,200])
plt.grid(False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.savefig(str(np.round(time.time())) + '_Delay_Tempos.png', dpi=600, bbox_inches='tight')


# plt.figure()
# sns.boxplot(x='TargetLateral',y='Lag',hue='height', data=df_metrics_all, palette='Blues')
# plt.title('Lag', fontsize=titleFontSize)
# plt.xlabel('Target Group')
# plt.ylabel('Lag (ms)')
# # plt.grid(False)
# plt.savefg(str(np.round(time.time())) + '_Lag.png', dpi=600, bbox_inches='tight')


### Two-Way Repeated Measures ANOVA

In [ ]:
detailing = False

# print('\n>> **Lateral has sig. effect \n>> Frontal no effect and \n>> **and there is a sig. interaction between lateral and frontal.\n')

res5 = pg.rm_anova(dv='EndError_cleaned', within=['TargetLateral', 'TargetVertical'], 
                     subject='PtxID', data=df_metrics_all, detailed=detailing)
print('\n EndError_cleaned LT\n', res5.round(3))

#-------------------------------------------------------------------------------------------
print('\nEndError on Height and Tempo')
# print('\n>> **Heigh has sig. effect \n>> Tempo no effect and \n>> **and there is a sig. interaction between lateral and frontal.\n')

res5 = pg.rm_anova(dv='EndError_cleaned', within=['height', 'tempos'], 
                     subject='PtxID', data=df_metrics_all, detailed=detailing)
print('\n EndError_cleaned LT\n', res5.round(3))

#-------------------------------------------------------------------------------------------
print('\nPathoffset on Target Location')
# print('\n>> Lateral has no effect \n>> Frontal no effect and \n>> **but there is a sig. interaction between lateral and frontal.\n')

res5 = pg.rm_anova(dv='PathOffsetNoLag_cleaned', within=['TargetLateral', 'TargetVertical'], 
                     subject='PtxID', data=df_metrics_all, detailed=detailing)
print('\n PathOffsetNoLag_cleaned LT\n', res5.round(3))

#-------------------------------------------------------------------------------------------
print('\nPathoffset on Height and Tempo')
# print('\n>> Height has no effect \n>> **Tempos have a sig. effect and \n>> **interaction between height and tempo.\n')

res5 = pg.rm_anova(dv='PathOffsetNoLag_cleaned', within=['height', 'tempos'], 
                     subject='PtxID', data=df_metrics_all, detailed=detailing)
print('\n PathOffsetNoLag_cleaned LT\n', res5.round(3))

#-------------------------------------------------------------------------------------------
# print('\nDelay on Height and Tempo')
# # print('\n>> Lag no effect \n>> Tempo no effect and \n>> No interaction between lateral and frontal.\n')

res6 = pg.rm_anova(dv='Lag', within=['tempos', 'height'], 
                  subject='PtxID', data=df_metrics_all,  detailed=detailing)
print('\n Lag HT\n', res6.round(3))

res7 = pg.rm_anova(dv='Lag', within=['TargetLateral', 'height'], 
                  subject='PtxID', data=df_metrics_all,  detailed=detailing)
print('\n Lag HL\n', res6.round(3))

# print('\nDelay on Lateral and Vertical Target Loc')
# # print('\n>> Lag no effect \n>> Tempo no effect and \n>> No interaction between lateral and frontal.\n')

# res6 = pg.rm_anova(dv='Lag', within=['TargetLateral', 'TargetVertical'], 
#                   subject='PtxID', data=df_metrics_all,  detailed=detailing)
# print('\n Lag HT\n', res6.round(3))


In [ ]:
import researchpy as rp
import statsmodels.api as sm
import scipy.stats as stats

In [ ]:
rp.summary_cont(df_metrics_all.groupby(["height", "tempos", "TargetLateral", "TargetVertical"])["EndError_cleaned"])

In [ ]:
plt.figure()
boxplot = df_metrics_all.boxplot(["EndError_cleaned"], by = ["height", "tempos", "TargetLateral", "TargetVertical"],
                     figsize = (16, 9),
                     showmeans = True,
                     notch = True)

boxplot.set_xlabel("Conditions")
boxplot.set_ylabel("Error (m)")

plt.savefig("boxplot.png")


In [ ]:
print('PtxID shape: ', df_metrics_all["PtxID"].shape)
print('TargetLateral shape', df_metrics_all["TargetLateral"].shape)
print('TargetVertical shape: ', df_metrics_all["TargetVertical"].shape)
print('tempos shape: ', df_metrics_all["tempos"].shape)
print('EndError_cleaned shape: ', df_metrics_all["EndError_cleaned"].shape)

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
model = smf.mixedlm("EndError ~ C(height) + tempos + C(TargetID)",
                    data = df_metrics_all, 
                    groups= df_metrics_all["PtxID"])

model_fit = model.fit()
print(model_fit.summary())

In [ ]:
print('\nEndError on Target Location')
mask = df_metrics_all['TargetLateral'] == 'center'
print('Average Error Center: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))
mask = df_metrics_all['TargetLateral'] == 'left'
print('Average Error Left: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))
mask = df_metrics_all['TargetLateral'] == 'right'
print('Average Error Right: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))

print('\nEndError on Target Vertical')
mask = df_metrics_all['TargetVertical'] == 'middle'
print('Average Error Middle: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))
mask = df_metrics_all['TargetVertical'] == 'far'
print('Average Error Far: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))
mask = df_metrics_all['TargetVertical'] == 'close'
print('Average Error Close: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))

print('\nError on Tempo')
# mask = df_metrics_all['tempos'] == '80'
print('Average Error on Tempo: ', np.round(df_metrics_all['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all['EndError_cleaned'].std(),2))
# mask = df_metrics_all['tempos'] == '120'
# print('Average Error 120bmp: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))
# mask = df_metrics_all['tempos'] == '160'
# print('Average Error 160bmp: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))

print('\nError on Height')
mask = (df_metrics_all['height'] == 'low') & (df_metrics_all['height'] == 'mid') & (df_metrics_all['height'] == 'high')
print('Average Error on Height: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))
# mask = df_metrics_all['height'] == 'mid'
# print('Average Error Mid: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))
# mask = df_metrics_all['height'] == 'high'
# print('Average Error High: ', np.round(df_metrics_all[mask]['EndError_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['EndError_cleaned'].std(),2))


print('\nPathOffset on Target Location')
mask = df_metrics_all['TargetLateral'] == 'center'
print('Average Offset Center: ', np.round(df_metrics_all['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))
# mask = df_metrics_all['TargetLateral'] == 'left'
# print('Average Offset Left: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))
# mask = df_metrics_all['TargetLateral'] == 'right'
# print('Average Offset Right: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))

print('\nPathOffset on Target Vertical')
mask = df_metrics_all['TargetVertical'] == 'middle'
print('Average Offset Middle: ', np.round(df_metrics_all['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))
# mask = df_metrics_all['TargetVertical'] == 'far'
# print('Average Offset Far: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))
# mask = df_metrics_all['TargetVertical'] == 'close'
# print('Average Offset Close: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))

print('\nOffset on Tempo')
mask = df_metrics_all['tempos'] == '80'
print('Average Offset 80bmp: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))
mask = df_metrics_all['tempos'] == '120'
print('Average Offset 120bmp: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))
mask = df_metrics_all['tempos'] == '160'
print('Average Offset 160bmp: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))

print('\nOffset on Height')
mask = df_metrics_all['height'] == 'low'
print('Average Offset Low: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))
mask = df_metrics_all['height'] == 'mid'
print('Average Offset Low: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))
mask = df_metrics_all['height'] == 'high'
print('Average Offset Low: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].mean(),2), ' cm', ' SD: ', np.round(df_metrics_all[mask]['PathOffsetNoLag_cleaned'].std(),2))



print('\nLag on Tempo')
mask = (df_metrics_all['tempos'] == '80') & (df_metrics_all['Lag'] < 200)
print('Average Lag 80bmp: ', np.round(df_metrics_all[mask]['Lag'].mean(),2), ' ms', ' SD: ', np.round(df_metrics_all[mask]['Lag'].std(),2))
mask = (df_metrics_all['tempos'] == '120') & (df_metrics_all['Lag'] < 200)
print('Average Lag 120bmp: ', np.round(df_metrics_all[mask]['Lag'].mean(),2), ' ms', ' SD: ', np.round(df_metrics_all[mask]['Lag'].std(),2))
mask = (df_metrics_all['tempos'] == '160') & (df_metrics_all['Lag'] < 200)
print('Average Lag 160bmp: ', np.round(df_metrics_all[mask]['Lag'].mean(),2), ' ms', ' SD: ', np.round(df_metrics_all[mask]['Lag'].std(),2))

mask = (df_metrics_all['Lag'] < 200)
print('\nAverage Overall Delay: ', np.round(df_metrics_all[mask]['Lag'].mean(),2),' ms')
print('Delay SD: ', np.round(df_metrics_all[mask]['Lag'].std(),2),' ms')

In [ ]:
detailing = False

res1 = pg.rm_anova(dv='EndError_cleaned', within=['height', 'tempos'], 
                  subject='PtxID', data=df_metrics_all,  detailed=detailing)
print('\n EndError HT\n', res1.round(3))

res2 = pg.rm_anova(dv='EndError_cleaned', within=['TargetLateral', 'TargetVertical'], 
                  subject='PtxID', data=df_metrics_all,  detailed=detailing)
print('\n EndError LV\n', res2.round(3))

res3 = pg.rm_anova(dv='PathOffset_cleaned', within=['height', 'tempos'], 
                  subject='PtxID', data=df_metrics_all,  detailed=detailing)
print('\n PathOffset_cleaned HT\n', res3.round(3))

res4 = pg.rm_anova(dv='PathOffset_cleaned', within=['TargetLateral', 'TargetVertical'], 
                  subject='PtxID', data=df_metrics_all,  detailed=detailing)
print('\n PathOffset_cleaned LV\n', res4.round(3))

res5 = pg.rm_anova(dv='PathOffset_cleaned', within=['TargetLateral', 'tempos'], 
                     subject='PtxID', data=df_metrics_all, detailed=detailing)
print('\n PathOffset_cleaned LT\n', res5.round(3))

res5 = pg.rm_anova(dv='PathOffsetNoLag_cleaned', within=['TargetLateral', 'tempos'], 
                     subject='PtxID', data=df_metrics_all, detailed=detailing)
print('\n PathOffsetNoLag_cleaned LT\n', res5.round(3))

res6 = pg.rm_anova(dv='Lag', within=['tempos', 'height'], 
                  subject='PtxID', data=df_metrics_all,  detailed=detailing)
print('\n Lag HT\n', res6.round(3))

# res7 = pg.rm_anova(dv='Lag_cleaned', within=['tempos', 'height'], 
#                   subject='PtxID', data=df_metrics_all,  detailed=True)
# print('\n Lag_cleaned HT Cleaned\n', res7.round(3))

# print('power: %.4f' % power_rm_anova(eta=0.861, m=2, n=5))

# print('n: %.4f' % power_rm_anova(eta=0.861, m=2, power=0.8))

# print('eta: %.4f' % power_rm_anova(n=5, m=2, power=0.95, alpha=0.05))


### Post hoc (pairwise t-tests) w/o correction

In [ ]:
# res2 = pg.rm_anova(dv='EndError_cleaned', within=['TargetLateral', 'TargetVertical'], 
#                   subject='PtxID', data=df_metrics_all,  detailed=detailing)
# print('\n EndError LV\n', res2.round(3))

posthocs = pg.pairwise_ttests(dv='EndError_cleaned', within=['height', 'TargetID'], 
                              subject='PtxID', data=df_metrics_all)
# pg.print_table(posthocs)

In [ ]:
df_metrics_all.groupby(['TargetLateral', 'TargetVertical'])['EndError_cleaned'].agg(['mean', 'std']).round(2)

In [ ]:
posthocs

### Path offset post-hoc analysis

In [ ]:
posthocs2 = pg.pairwise_ttests(dv='PathOffsetNoLag_cleaned', within=['TargetLateral', 'TargetVertical'], 
                              subject='PtxID', data=df_metrics_all)

In [ ]:
df_metrics_all.groupby(['TargetLateral', 'TargetVertical'])['PathOffsetNoLag_cleaned'].agg(['mean', 'std']).round(2)

In [ ]:
posthocs3 = pg.pairwise_ttests(dv='PathOffsetNoLag_cleaned', within=['height', 'tempos'], 
                              subject='PtxID', data=df_metrics_all)

In [ ]:
posthocs3

In [ ]:
# result  = df_metrics_all.to_json(orient="split")

# f = open("jsondataframe.txt", "a")
# f.write(result)
# f.close()

In [ ]:
mask = (df_metrics_all['tempos'] == '80') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['MaxVel_Real'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['MaxVel_Real'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
xVals = np.arange(0,5)

plt.errorbar(xVals, ptxAvMaxVels, SE, color='r')
plt.plot(ptxAvMaxVels,'ko',ms=12)
plt.xlim([-0.5,5.5])
plt.xlabel('Participants')
plt.ylim([-1,5])
plt.ylabel('Peak Velocity (ms-1)')

mask = (df_metrics_all['tempos'] == '120') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['MaxVel_Real'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['MaxVel_Real'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
plt.errorbar(xVals, ptxAvMaxVels, SE, color='g')
plt.plot(ptxAvMaxVels,'ko',ms=12)

mask = (df_metrics_all['tempos'] == '160') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['MaxVel_Real'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['MaxVel_Real'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
plt.errorbar(xVals, ptxAvMaxVels, SE, color='b')
plt.plot(ptxAvMaxVels,'ko',ms=12)

plt.legend(['80', '120', '160'])

In [ ]:
mask = (df_metrics_all['tempos'] == '80') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['PathOffsetNoLag_cleaned'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['PathOffsetNoLag_cleaned'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
xVals = np.arange(0,5)

# print('SD: ', len(ptxSDMaxVels), ' \nSE: ', len(SE), ' \nxVals: ', len(xVals))

plt.errorbar(xVals, ptxAvMaxVels, SE, color='r')
plt.legend(['80'])
plt.plot(ptxAvMaxVels,'ko',ms=12)
plt.xlim([-0.5,5])
plt.xlabel('Participants')
# plt.ylim([-1,5])
plt.ylabel('Path Offset (cm)')

mask = (df_metrics_all['tempos'] == '120') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['PathOffsetNoLag_cleaned'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['PathOffsetNoLag_cleaned'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
plt.errorbar(xVals, ptxAvMaxVels, SE, color='g')
plt.plot(ptxAvMaxVels,'ko',ms=12)

mask = (df_metrics_all['tempos'] == '160') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['PathOffsetNoLag_cleaned'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['PathOffsetNoLag_cleaned'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
plt.errorbar(xVals, ptxAvMaxVels, SE, color='b')
plt.plot(ptxAvMaxVels,'ko',ms=12)

# plt.legend(['80', '120', '160'])

plt.ylim([2,7])

In [ ]:
mask = (df_metrics_all['tempos'] == '80') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['EndError_cleaned'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['EndError_cleaned'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
xVals = np.arange(0,5)
plt.errorbar(xVals, ptxAvMaxVels, SE, color='r')
plt.plot(ptxAvMaxVels,'ko',ms=12)
plt.xlim([-0.5,5])
plt.xlabel('Participants')
# plt.ylim([-1,5])
plt.ylabel('EndError (cm)')

mask = (df_metrics_all['tempos'] == '120') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['EndError_cleaned'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['EndError_cleaned'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
plt.errorbar(xVals, ptxAvMaxVels, SE, color='g')
plt.plot(ptxAvMaxVels,'ko',ms=12)

mask = (df_metrics_all['tempos'] == '160') & (df_metrics_all['height'] == 'low') 
ptxAvMaxVels = df_metrics_all[mask].groupby('PtxID')['EndError_cleaned'].mean()
ptxSDMaxVels = df_metrics_all[mask].groupby('PtxID')['EndError_cleaned'].std()
SE = ptxSDMaxVels / np.sqrt(len(ptxSDMaxVels))
plt.errorbar(xVals, ptxAvMaxVels, SE, color='b')
plt.plot(ptxAvMaxVels,'ko',ms=12)

# plt.legend(['80', '120', '160'])

# plt.ylim([-1,15])

# print('SD: ', len(ptxSDMaxVels), ' \nSE: ', len(SE), ' \nxVals: ', len(xVals))


In [ ]:
ptxIds = pd.unique(df_metrics_all['PtxID'])
print('Participants: ', ptxIds)

In [ ]:
plt.figure()

plt.subplot(1,2,1)
sns.barplot(x='tempos', y = 'PathOffsetNoLag_cleaned', data=df_metrics_all)
plt.title('Path Offset (cm)')
plt.xlabel('Tempo')
# plt.ylim([0, 75])

plt.subplot(1,2,2)
sns.barplot(x='tempos', y = 'EndError_cleaned', data=df_metrics_all)
# plt.ylim([0, 10])
plt.title('Target Hit Error (cm)')
plt.xlabel('Tempo')

# plt.subplot(1,4,3)
# sns.barplot(x='tempos', y = 'Lag', data=df_metrics_all)
# plt.ylim([0, 100])
# plt.title('Target Hit Error (cm)')
# plt.xlabel('Tempo')

# plt.subplot(1,4,4)
# sns.barplot(x='tempos', y = 'MaxVel_Virt', palette='Blues', data=df_metrics_all)
# sns.barplot(x='tempos', y = 'MaxVel_Real', palette='Greens', data=df_metrics_all)
# plt.ylim([0, 10])
# plt.title('Target Hit Error (cm)')
# plt.xlabel('Tempo')
plt.legend(['Virtual','Real'])

In [ ]:
# print('Participants: ', ptxIds)
# print('Heights: ', pd.unique(df_all['height']))
# print('Tempos: ', pd.unique(df_all['tempos']))

# ptxNum = 0
# h_mask_low = (df_all['tempos'] == '80') & (df_all['PtxID'] == ptxIds[ptxNum])
# h_mask_mid = (df_all['tempos'] == '120') & (df_all['PtxID'] == ptxIds[ptxNum])
# h_mask_hig = (df_all['tempos'] == '160') & (df_all['PtxID'] == ptxIds[ptxNum])
# h_mask_nh = (df_all['tempos'] == 'noTempo') & (df_all['PtxID'] == ptxIds[ptxNum])

# print('\n Ptx name: ', ptxIds[ptxNum])
# print('Num of 80 trials: ' , len(df_all[h_mask_low]['TrialNum'].values))
# print('Num of 120 trials: ' , len(df_all[h_mask_mid]['TrialNum'].values))
# print('Num of 160 trials: ' , len(df_all[h_mask_hig]['TrialNum'].values))
# print('Num of no tempo trials: ' , len(df_all[h_mask_nh]['TrialNum'].values))

#@title Default title text
# TO-DOs
- [x] Make sure rotations do not jumpt back to zero from 360 i.e. cap rotations to a specific region 
- [x] Create a nice data frame with all the values in the correct format, and add more parameters to the dataframe
- [x] Create average plots with standard deviation as shaded area


- [ ] In "ComputeErrors" function implement more extensive data frame with
  - [ ] Participant ID
  - [ ] Target ID and location 
  - [ ] Height, tempo and trial number 

In [ ]:
masktemp = (df_all['gameObjectName'] == 'realFingerTip')
trialsofinterest = pd.unique(df_all[masktemp].TrialNum)
    
ptxMask = (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['TrialNum'] == trialsofinterest[5])
ptx = pd.unique(df_all[ptxMask]['PtxID'].values)[1]
ptx

# ptx = 'Poppy' 

In [ ]:
ptx = pd.unique(df_all['PtxID'])
ptxNum = 3
print('Participant: ', ptx[ptxNum])

mask = (df_all['PtxID'] == ptx[ptxNum]) & (df_all['height'] == 'high') & (df_all['tempos'] == '120')& (df_all['targetID'] == 'row_C3')  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
real_mask = (df_all['PtxID'] == ptx[ptxNum]) & (df_all['height'] == 'high') & (df_all['tempos'] == '120')& (df_all['targetID'] == 'row_C3')  & (df_all['gameObjectName'] == 'realFingerTip')

plt.plot(df_all[real_mask]['xPos'].values[-1-150:-1], df_all[real_mask]['zPos'].values[-1-150:-1],'k-o')
plt.plot(df_all[mask]['xPos'].values[-1-150:-1], df_all[mask]['zPos'].values[-1-150:-1],'m-o')

plt.title('Example Reaching Trial')
plt.ylabel('Z-Axis / m')
plt.xlabel('X-Axis / m')
plt.legend(['Real','Virtual'])
# plt.xlim([-1.175, -0.8])
# plt.ylim([0, 0.55])

plt.savefig(ptx[ptxNum] + "_" + str(np.round(time.time())) + '_ExampleReach.png', dpi=600, bbox_inches='tight')

In [ ]:
ptx = 'Susan'
mask = (df_all['PtxID'] == ptx[ptxNum]) & (df_all['height'] == 'low') & (df_all['tempos'] == '80')  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
realmask = (df_all['PtxID'] == ptx[ptxNum]) & (df_all['height'] == 'low') & (df_all['tempos'] == '80')  & (df_all['gameObjectName'] == 'realFingerTip')

plt.plot(df_all[realmask]['xPos'].values[-1-1500:-1], df_all[realmask]['zPos'].values[-1-1500:-1],'k-o')
plt.plot(df_all[mask]['xPos'].values[-1-1500:-1], df_all[mask]['zPos'].values[-1-1500:-1],'m-o', ms = 3)

plt.title('Example Reaches')
plt.ylabel('Z-Axis / m')
plt.xlabel('X-Axis / m')

plt.savefig(str(np.round(time.time())) + '_ExampleReaches.png', dpi=600, bbox_inches='tight')

In [ ]:
mask_1 = (df_all['PtxID'] == ptx[ptxNum]) & (df_all['height'] == 'mid') & (df_all['tempos'] == '120')& (df_all['targetID'] == 'row_A1')  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
trials = pd.unique(df_all[mask_1]['TrialNum'])

if len(trials) < 1:
    mask_1 = (df_all['PtxID'] == ptx[ptxNum]) & (df_all['height'] == 'noHeight') & (df_all['tempos'] == '120')& (df_all['targetID'] == 'row_A1')  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
    trials = pd.unique(df_all[mask_1]['TrialNum'])

trials

# mask_2 = (df_all['TrialNum'] == trials[5]) & (df_all['PtxID'] == 'Davide') & (df_all['height'] == 'noHeight') & (df_all['tempo'] == '120')& (df_all['targetID'] == 'row_A1')  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
# plt.plot(df_all[mask_2]['zPos'],'r-')

In [ ]:
### Save resultant dataframe data to csv

compression_opts = dict(method='zip',
                        archive_name='df_all.csv')  
# df_all.to_csv('df_all.zip', index=False,
#           compression=compression_opts)  

df_all.to_csv('df_all.csv', index=False)  

In [ ]:
df_metrics_all.head(2)

# Linear Mixed Effects Model Analysis

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

In [ ]:
data = df_metrics_all.dropna()

In [ ]:
plt.rcParams["figure.figsize"] = (8,5)
sns.set_theme(style="whitegrid")

In [ ]:
# Sort by target id
data_sort = data.sort_values(by = 'TargetID')

sns.set()
ax = sns.pointplot(data=data_sort, x='TargetID', y='EndError_cleaned',hue='height', dodge=True,
              capsize=.1, errwidth=1, palette='rocket')

plt.xticks(rotation=45)

# plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.xlabel('Target IDs', fontsize=14)
plt.ylabel('Error (cm)', fontsize=14)
plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
           ['A1','A2','A3','A4','A5','A6','B1','B2','B3','B4','B5','B6','C1','C2','C3','C4','C5','C6'])

ax.set_facecolor((0.95,0.95,0.95))
sns.set_style("whitegrid")
plt.title('Positional Error',fontsize=16.5)

plt.savefig(str(np.round(time.time())) + '_Error_IP.png', dpi=600, bbox_inches='tight')

In [ ]:
# Sort by target id
data_sort = data.sort_values(by = 'TargetID')

sns.set()
ax = sns.pointplot(data=data_sort, x='TargetID', y='Lag',hue='height', dodge=True,
              capsize=.1, errwidth=1, palette='rocket')

plt.xticks(rotation=45)

# plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.xlabel('Target IDs', fontsize=14)
plt.ylabel('Tempo (bpm)', fontsize=14)
plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
           ['A1','A2','A3','A4','A5','A6','B1','B2','B3','B4','B5','B6','C1','C2','C3','C4','C5','C6'])

ax.set_facecolor((0.95,0.95,0.95))
sns.set_style("whitegrid")
plt.title('Tempos',fontsize=16.5)

plt.savefig(str(np.round(time.time())) + '_Lag_IP.png', dpi=600, bbox_inches='tight')

In [ ]:
mask = (data_sort['height'] == 'low') & (data_sort['TargetLateral'] == 'left')
left_low = data_sort[mask]['EndError_cleaned'].mean()

mask = (data_sort['height'] == 'low') & (data_sort['TargetLateral'] == 'right')
right_low = data_sort[mask]['EndError_cleaned'].mean()

mask = (data_sort['height'] == 'low') & (data_sort['TargetLateral'] == 'center')
center_low = data_sort[mask]['EndError_cleaned'].mean()



mask = (data_sort['height'] == 'mid') & (data_sort['TargetLateral'] == 'left')
left_mid = data_sort[mask]['EndError_cleaned'].mean()

mask = (data_sort['height'] == 'mid') & (data_sort['TargetLateral'] == 'right')
right_mid = data_sort[mask]['EndError_cleaned'].mean()

mask = (data_sort['height'] == 'mid') & (data_sort['TargetLateral'] == 'center')
center_mid = data_sort[mask]['EndError_cleaned'].mean()



mask = (data_sort['height'] == 'high') & (data_sort['TargetLateral'] == 'left')
left_high = data_sort[mask]['EndError_cleaned'].mean()

mask = (data_sort['height'] == 'high') & (data_sort['TargetLateral'] == 'right')
right_high = data_sort[mask]['EndError_cleaned'].mean()

mask = (data_sort['height'] == 'high') & (data_sort['TargetLateral'] == 'center')
center_high = data_sort[mask]['EndError_cleaned'].mean()

In [ ]:
peri_low = np.nanmean([left_low, right_low])
center_low

peri_mid = np.nanmean([left_mid, right_mid])
center_mid

peri_high = np.nanmean([left_high,right_high])
center_high

x = ['PeriLow', 'CenterLow', 'PeriMid', 'CenterMid', 'PeriHigh','CenterHigh']
y = [peri_low, center_low, peri_mid, center_mid, peri_high, center_high]

In [ ]:
vals = list(zip(x,y)) # Create a two column matrix 
error_data_av = pd.DataFrame(vals, columns =['X','Y'])
error_data_av

In [ ]:
y_center = [center_low, center_mid, center_high]
y_peri = [peri_low, peri_mid, peri_high]

print('M center: ', np.nanmean(y_center), ' SD: ', np.nanstd(y_center))
print('M peri: ', np.nanmean(y_peri), ' SD: ', np.nanstd(y_peri))

In [ ]:
# sns.boxplot(x = ['PeriLow', 'CenterLow', 'PeriMid', 'CenterMid', 'PeriHigh','CenterHigh'], y= y)

plt.figure()
sns.barplot(x='X',y='Y', data=error_data_av, palette='Blues')
plt.title('Error', fontsize=titleFontSize)
plt.xlabel('Positions')
plt.ylabel('Error (cm)')
# plt.ylim([-10,200])
plt.grid(False)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.savefig(str(np.round(time.time())) + '_Delay_Tempos.png', dpi=600, bbox_inches='tight')

In [ ]:
# # Sort by target id
# data_sort = data.sort_values(by = 'TargetID')
# # sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

# sns.set()
# sns.pointplot(data=data_sort, x='TargetID', y='PathOffsetNoLag_cleaned',hue='height', dodge=True,
#               capsize=.1, errwidth=1, palette='colorblind')

# plt.xticks(rotation=45)

# plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.xlabel('Target IDs')
# plt.ylabel('Offset (cm)')
# plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],['A1','A2','A3','A4','A5','A6',
#                                                          'B1','B2','B3','B4','B5','B6',
#                                                          'C1','C2','C3','C4','C5','C6'])

# ax.set_facecolor((0.95,0.95,0.95))
# sns.set_style("whitegrid")

# plt.savefig(str(np.round(time.time())) + '_PathOffset_TargID_Height_IP.png', dpi=600, bbox_inches='tight')

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)
sns.set_theme(style="whitegrid")

In [ ]:
# Sort by target id
#---------------------------------------------------------------------------------------------------------
#------------------------------------- Path offset tempos  ------------------------------------------
#---------------------------------------------------------------------------------------------------------
plt.subplot(121)
plt.tight_layout(pad=3.0)
data_sort = data.sort_values(by = 'TargetID')
# sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

sns.set()
ax = sns.pointplot(data=data_sort, x='TargetID', y='PathOffsetNoLag_cleaned',hue='tempos', dodge=True,
              capsize=.1, errwidth=1, palette='rocket')

plt.xticks(rotation=45)

# plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.xlabel('Target IDs', fontsize=14)
plt.ylabel('Offset (cm)', fontsize=14)
plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],['A1','A2','A3','A4','A5','A6',
                                                         'B1','B2','B3','B4','B5','B6',
                                                         'C1','C2','C3','C4','C5','C6'])
plt.title('Path Offset > TargetID - Tempo',fontsize=16.5)
ax.set_facecolor((0.95,0.95,0.95))
sns.set_style("whitegrid")

# plt.savefig(str(np.round(time.time())) + '_PathOffset_TargID_Tempo_IP.png', dpi=600, bbox_inches='tight')

#---------------------------------------------------------------------------------------------------------
#------------------------------------- Path offset heights ------------------------------------------
#---------------------------------------------------------------------------------------------------------

plt.subplot(122)
plt.tight_layout(pad=3.0)
data_sort = data.sort_values(by = 'TargetID')
# sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

sns.set()
ax = sns.pointplot(data=data_sort, x='tempos', y='PathOffsetNoLag_cleaned',hue='height', dodge=True,
              capsize=.1, errwidth=1, palette='rocket')

plt.xticks(rotation=45)

# plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.xlabel('Tempo', fontsize=14)
plt.ylabel('Offset (cm)', fontsize=14)
# plt.xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],['A1','A2','A3','A4','A5','A6',
#                                                          'B1','B2','B3','B4','B5','B6',
#                                                          'C1','C2','C3','C4','C5','C6'])
plt.title('Path Offset > Tempo - Heights',fontsize=16.5)
ax.set_facecolor((0.95,0.95,0.95))
sns.set_style("whitegrid")

plt.savefig(str(np.round(time.time())) + '_PathOffset_TargID_Tempo_Height_IP.png', dpi=600, bbox_inches='tight')

In [ ]:
mask = (data_sort['tempos'] == 80) & (data_sort['height'] == 'low') 
mean_80_low = data_sort[mask]['PathOffsetNoLag_cleaned']

mask = mask = (data_sort['tempos'] == 120) & (data_sort['height'] == 'low') 
mean_120_low = data_sort[mask]['PathOffsetNoLag_cleaned']

mask = mask = (data_sort['tempos'] == 160) & (data_sort['height'] == 'low') 
mean_160_low = data_sort[mask]['PathOffsetNoLag_cleaned']

mask = (data_sort['tempos'] == 80) & (data_sort['height'] == 'mid') 
mean_80_mid = data_sort[mask]['PathOffsetNoLag_cleaned']

mask = mask = (data_sort['tempos'] == 120) & (data_sort['height'] == 'mid') 
mean_120_mid = data_sort[mask]['PathOffsetNoLag_cleaned']

mask = mask = (data_sort['tempos'] == 160) & (data_sort['height'] == 'mid') 
mean_160_mid = data_sort[mask]['PathOffsetNoLag_cleaned']

mask = (data_sort['tempos'] == 80) & (data_sort['height'] == 'high') 
mean_80_high = data_sort[mask]['PathOffsetNoLag_cleaned']

mask = mask = (data_sort['tempos'] == 120) & (data_sort['height'] == 'high') 
mean_120_high = data_sort[mask]['PathOffsetNoLag_cleaned']

mask = mask = (data_sort['tempos'] == 160) & (data_sort['height'] == 'high') 
mean_160_high = data_sort[mask]['PathOffsetNoLag_cleaned']



In [ ]:
m_80_low = np.nanmean(mean_80_low)
sd_80_low = np.nanstd(mean_80_low)
m_120_low = np.nanmean(mean_120_low)
sd_120_low = np.nanstd(mean_120_low)
m_160_low = np.nanmean(mean_160_low)
sd_160_low = np.nanstd(mean_160_low)
m_80_mid = np.nanmean(mean_80_mid)
sd_80_mid = np.nanstd(mean_80_mid)
m_120_mid = np.nanmean(mean_120_mid)
sd_120_mid = np.nanstd(mean_120_mid)
m_160_mid = np.nanmean(mean_160_mid)
sd_160_mid = np.nanstd(mean_160_mid)
m_80_high = np.nanmean(mean_80_high)
sd_80_high = np.nanstd(mean_80_high)
m_120_high = np.nanmean(mean_120_high)
sd_120_high = np.nanstd(mean_120_high)
m_160_high = np.nanmean(mean_160_high)
sd_160_high = np.nanstd(mean_160_high)

In [ ]:
print('Mean 80 low:      ', np.round(m_80_low,2), '    SD 80 low:   ', np.round(sd_80_low,2))
print('Mean 120 low:      ', np.round(m_120_low,2), '    SD 80 low:   ', np.round(sd_120_low,2))
print('Mean 160 low:      ', np.round(m_160_low,2), '    SD 80 low:   ', np.round(sd_160_low,2))

print('Mean 80 mid:      ', np.round(m_80_mid,2), '    SD 80 mid:   ', np.round(sd_80_mid,2))
print('Mean 120 mid:      ', np.round(m_120_mid,2), '    SD 120 mid:   ', np.round(sd_120_mid,2))
print('Mean 160 mid:      ', np.round(m_160_mid,2), '    SD 160 mid:   ', np.round(sd_160_mid,2))

print('Mean 80 high:      ', np.round(m_80_high,2), '    SD 80 high:   ', np.round(sd_80_high,2))
print('Mean 120 high:      ', np.round(m_120_high,2), '    SD 120 high:   ', np.round(sd_120_high,2))
print('Mean 160 high:      ', np.round(m_160_high,2), '    SD 160 high:   ', np.round(sd_160_high,2))

In [ ]:
data["tempos"] = data["tempos"].astype(str)
data["tempos"].values[0]

In [ ]:
data["TrialNum"] = data["TrialNum"].astype(str)
data["TrialNum"].values[0]

In [ ]:
data.head()

In [ ]:
data.to_csv('QuestData_Metrics2.csv')

In [ ]:
# model1 = smf.ols(formula='EndError_cleaned ~ (tempos * height * TargetID)', data=data).fit() 
# model1.summary()

# model = smf.mixedlm("EndError_cleaned ~ (TargetID * height)", data, groups=data['PtxID'])
# res = model.fit() # method=["lbfgs"]
# res.summary()

In [ ]:
# md = smf.mixedlm("EndError_cleaned ~ height * TargetID", data, groups=data["PtxID"])
# mdf = md.fit()
# mdf.summary()

# Distortion Analysis
### For each height and tempo create a dataframe with all the movement end-points for each target for the distortion analysis 

In [ ]:
targetID = pd.unique(df_all['targetID'])
participants = pd.unique(df_all['PtxID'])

height = 'low'
tempo = '120'
trow = 'B'

df_terminal_pos = None 

for p in participants:
    
    target = []

    for t in targetID: # Go through all the targets but ...

        if trow in t: # ... only select one specific row 
            
            
#           mask_virtualfinger = (df_all['height'] == height) & (df_all['tempo'] == tempo)  & (df_all['PtxID'] == p)  & (df_all['targetID'] == t)  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
#           mask_realfinger = (df_all['height'] == height) & (df_all['tempo'] == tempo) & (df_all['PtxID'] == p)  & (df_all['targetID'] == t)  & (df_all['gameObjectName'] == 'realFingerTip')

            Row_A_point_x = []
            Row_A_point_z = []
            ste_x = []
            ste_z = []

            vRow_A_point_x = []
            vRow_A_point_z = []
            vste_x = []
            vste_z = []
            
            target.append(t)


            mask_1 = (df_all['PtxID'] == p) & (df_all['height'] == height) & (df_all['tempos'] == tempo) & (df_all['targetID'] == t)  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
            trials = pd.unique(df_all[mask_1]['TrialNum'])
            
            if len(trials) < 1:
                mask_1 = (df_all['PtxID'] == p) & (df_all['height'] == 'noHeight') & (df_all['tempos'] == tempo) & (df_all['targetID'] == t)  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
                trials = pd.unique(df_all[mask_1]['TrialNum'])
            
            for tr in trials:
                
                mask_2_virt = (df_all['TrialNum'] == tr) & (df_all['PtxID'] == p) & (df_all['height'] == height) & (df_all['tempos'] == tempo)& (df_all['targetID'] == t)
                
                # Virtual finger tip terminal position
                end_points_x = df_all[mask_2_virt & (df_all['gameObjectName'] == 'r_index_fingernail_marker')]['xPos'].values[-1-30:-1]
                end_points_z = df_all[mask_2_virt & (df_all['gameObjectName'] == 'r_index_fingernail_marker')]['zPos'].values[-1-30:-1]
                vRow_A_point_x.append(np.round(np.nanmean(end_points_x, axis=0),5))
                vste_x.append(np.round(np.nanstd(end_points_x) / np.sqrt(len(end_points_x)), 5))
                vRow_A_point_z.append(np.round(np.nanmean(end_points_z, axis=0),5))
                vste_z.append(np.round(np.nanstd(end_points_z) / np.sqrt(len(end_points_z)), 5))

                # Real finger tip terminal position            
                end_points_x = df_all[mask_2_virt & (df_all['gameObjectName'] == 'realFingerTip')]['xPos'].values[-1-30:-1]
                end_points_z = df_all[mask_2_virt & (df_all['gameObjectName'] == 'realFingerTip')]['zPos'].values[-1-30:-1]
                Row_A_point_x.append(np.round(np.nanmean(end_points_x, axis=0),5))
                ste_x.append(np.round(np.nanstd(end_points_x) / np.sqrt(len(end_points_x)), 5))
                Row_A_point_z.append(np.round(np.nanmean(end_points_z, axis=0),5))
                ste_z.append(np.round(np.nanstd(end_points_z) / np.sqrt(len(end_points_z)), 5))
                
            # Average across the trials 
            vRow_A_point_x = np.nanmean(vRow_A_point_x, axis=0)
            vste_x = np.nanmean(vste_x, axis=0)
            vRow_A_point_z = np.nanmean(vRow_A_point_z, axis=0)
            vste_z = np.nanmean(vste_z, axis=0)
            Row_A_point_x = np.nanmean(Row_A_point_x, axis=0)
            ste_x = np.nanmean(ste_x, axis=0)
            Row_A_point_z = np.nanmean(Row_A_point_z, axis=0)
            ste_z = np.nanmean(ste_z, axis=0)

#             dataList = zip(Row_A_point_x, Row_A_point_z, ste_x, ste_z, vRow_A_point_x , vRow_A_point_z, vste_x, vste_z, t)
#             tmpDF = pd.DataFrame(dataList, columns=['Real_X_Pos','Real_Z_Pos', 'Real_SE_X',  'Real_SE_Z', 'Virtual_X_Pos','Virtual_Z_Pos', 'Virtual_SE_X',  'Virtual_SE_Z', 'targetID'])    

            dataList = [Row_A_point_x, Row_A_point_z, ste_x, ste_z, vRow_A_point_x , vRow_A_point_z, vste_x, vste_z, t, p, height, tempo]
            tmpDF = pd.DataFrame([dataList], columns = ['Real_X_Pos', 'Real_Z_Pos', 'Real_X_SE','Real_Z_SE', 'Virt_X_Pos','Virt_Z_Pos','Virt_X_SE','Virt_Z_SE','TargetID','PtxID','Height','Tempo'])
#             tmpDF = pd.DataFrame.from_dict(dataDict.items())
#             tmpDF = pd.DataFrame.from_dict(dataDict.items(), orient = 'index').T
#             tmpDF = pd.DataFrame(list(dataDict.items()), orient = 'index', columns = ['Real_X_Pos', 'Real_Z_Pos','Virt_X_Pos','Virt_Z_Pos','Real_X_SE','Real_Z_SE','Virt_X_SE','Virt_Z_SE','TargetID','PtxID','Height','Tempo'])

            # Save info to dataframe
            if df_terminal_pos is None:
                df_terminal_pos = tmpDF
            else:
                df_terminal_pos = pd.concat((df_terminal_pos, tmpDF))

In [ ]:
targetID = pd.unique(df_all['targetID'])
print(targetID)
participants = pd.unique(df_all['PtxID'])
print(participants)
temps = pd.unique(df_all['tempos'])
print(temps)
hts = pd.unique(df_all['height'])
print(hts)

mask_1 = (df_all['PtxID'] == 'Pete') & (df_all['height'] == 'low') & (df_all['tempos'] == '80') & (df_all['targetID'] == 'row_B2')  & (df_all['gameObjectName'] == 'r_index_fingernail_marker')
trials = pd.unique(df_all[mask_1]['TrialNum'])
    
trials

In [ ]:
df_all

In [ ]:
# Sort indeces according to target id:
df_terminal_pos = df_terminal_pos.sort_values(by=['TargetID', 'PtxID'])

mymask = (df_terminal_pos['PtxID'] == ptx) 
plt.plot(df_terminal_pos[mymask]['TargetID'], df_terminal_pos[mymask]['Real_Z_Pos'],'r-o')
plt.plot(df_terminal_pos[mymask]['TargetID'], df_terminal_pos[mymask]['Virt_Z_Pos'],'g-o')
plt.legend(['Real','Virtual'])
plt.title('Virtual end-position distortion')
plt.ylabel('Position / m')
plt.xlabel('Target ID')

print('Participant: ', ptx)

In [ ]:
def undistort_point(undistortion_params,r_distorted):
    undistorted = r_distorted*(1 + undistortion_params[0] * r_distorted
                               + undistortion_params[1] * r_distorted**2
                               + undistortion_params[2] * r_distorted**3)
    return(undistorted)

def fun(undistortion_params,r_distorted, un_distorted):
    #Compute residuals.
    undistorted = undistort_point(undistortion_params, r_distorted)
#     return((undistorted - np.linspace(np.nanmean(r_distorted,axis=0),np.nanmean(r_distorted,axis=0),len(r_distorted)))).ravel()
    return((undistorted - un_distorted)).ravel()

In [ ]:
r_distorted = df_terminal_pos[mymask]['Virt_Z_Pos']
un_distorted = df_terminal_pos[mymask]['Real_Z_Pos']

x0 = np.zeros(3).ravel()
res = least_squares(fun, x0,  verbose=2, ftol=1e-12,loss='linear', args=([r_distorted, un_distorted]))

In [ ]:
xvalz = [0,1,2,3,4,5]
undistorted = undistort_point(res.x,r_distorted)    
plt.plot(xvalz, r_distorted,label='Oculus Quest',alpha=0.5,color='b',marker = 'o', ms = 10)
plt.plot(xvalz, undistorted,label='Corrected',alpha=0.5,color='g',marker = 'o', ms = 10,linewidth = 8)
plt.plot(xvalz, un_distorted,label='MoCap',alpha=0.85,color='r',marker = 'o', ms = 10)
# plt.plot(xvalz, np.linspace(np.nanmean(r_distorted,axis=0),np.nanmean(r_distorted[0],axis=0),len(r_distorted)),label='target',alpha=0.85,color='r')
plt.title('Positional Distortion Metric')
plt.xlabel('Target ID')
plt.ylabel('Forward Position / m')
plt.xlim([-1, 6])
plt.ylim([0.2, 0.4])
plt.xticks([-1,0,1,2,3,4,5,6],['','1','2','3','4','5','6',''])

plt.legend()
plt.savefig(ptx + "_" + str(np.round(time.time())) + '_DistortionMetric.png', dpi=600, bbox_inches='tight')

In [ ]:
print(np.round(res.x)) # 

# Old code

In [ ]:
def ComputeErrors2(df_in):
    masktemp = (df_in['gameObjectName'] == 'realFingerTip')
    adaptationTrialNumbers = pd.unique(df_in[masktemp].TrialNum)
    # np.random.shuffle(adaptationTrialNumbers)

    df_out = None # If arrays are to be saved use this 
    dat_List = []

    ptxes = pd.unique(df_in['PtxID'])
    
    for ptx in ptxes:
        print('Ptx: ', ptx)
        
        for i in range(len(adaptationTrialNumbers)):
            realFingerMask = (df_in['PtxID'] == ptx) & (df_in['gameObjectName'] == 'realFingerTip') & (df_in['TrialNum'] == adaptationTrialNumbers[i])
            virtualFingerMask = (df_in['PtxID'] == ptx) & (df_in['gameObjectName'] == 'r_index_fingernail_marker') & (df_in['TrialNum'] == adaptationTrialNumbers[i])
            ptxMask = (df_in['PtxID'] == ptx) & (df_in['gameObjectName'] == 'r_index_fingernail_marker') & (df_in['TrialNum'] == adaptationTrialNumbers[i])

            # timeMask = df.loc[(df['gameObjectName'] == 'realFingerTip') & (df['trialNumber'] == adaptationTrialNumbers[i]), ['time']]
            timeMask = df_in['TrialNum'] == adaptationTrialNumbers[i]

            try:

                plt.figure(1) # All positions
                plt.plot(df_in[realFingerMask].xPos, df_in[realFingerMask].zPos,'r')
                plt.plot(df_in[virtualFingerMask].xPos, df_in[virtualFingerMask].zPos,'g')
                plt.title('X-Z Position / m')
                plt.gca().set_aspect('equal', adjustable='box')
                plt.gca().set_aspect('equal', adjustable='box')
                plt.legend(['Real','Virtual'])

                ax = plt.figure(2)
                tangXZ_Real = np.sqrt(np.power(df_in[realFingerMask].xPos,2) + np.power(df_in[realFingerMask].zPos,2))
                tangXZ_Virt = np.sqrt(np.power(df_in[virtualFingerMask].xPos,2) + np.power(df_in[virtualFingerMask].zPos,2))

                print('Trial: ', i)

                try:
                    tangXZ_Real_Vel = np.abs(np.diff(savgol_filter(tangXZ_Real, 75, 4)))
                    tangXZ_Virt_Vel = np.abs(np.diff(savgol_filter(tangXZ_Virt, 75, 4)))


                    print('Past issue: ', i)

                    plt.plot(tangXZ_Real_Vel,'r')
                    plt.plot(tangXZ_Virt_Vel,'g')
                    plt.title('Lateral Velocity (x-z axis) $\mathregular{ms^{-1}}$')

                    # Sampling Frequency and Time
                    times = df_in[timeMask].time.tolist()
                    val = np.where(times == numpy.amin(times))
                    startTimeIdx = val[0][0]
                    # print('min: ', startTimeIdx)

                    print('Past 2nd issue: ', i)

                    plt.figure(3) 
                    plt.plot(times)
                    plt.plot(startTimeIdx, times[startTimeIdx], 'rx')
                    # print('Movement duration: ', times[-1], 's')
                    startMovIdx = int(np.round((startTimeIdx/10))) # Convert between time series and movement array by dividing by 10? 
                    sampleFreq = np.round(len(tangXZ_Real_Vel[startMovIdx:])/times[-1])
                    # print('Sampling Freq: ', sampleFreq)

                    print('Past 3rd issue: ', i)

                    #--------------- Cross-Correlation ---------------------------------------------
                    corr = np.correlate(tangXZ_Real_Vel - np.mean(tangXZ_Real_Vel), 
                                      tangXZ_Virt_Vel - np.mean(tangXZ_Virt_Vel),
                                      mode='full')
                    sampleDifference = np.argmax(tangXZ_Virt_Vel[20:]) - np.argmax(tangXZ_Real_Vel[20:])

                    print('Past correlation issue: ', i)

                    if sampleDifference > 50:
                        lag = (sampleDifference  * (1/sampleFreq)) * 1000
                        print('Lag is too large: ' , lag)
                    else: 
                        lag = (sampleDifference  * (1/sampleFreq)) * 1000
                        # print('Lag: ', np.round(lag), 'ms')

                        #--------------- Positional-Error ---------------------------------------------
                        # MSE = np.sum(np.power(np.abs((tangXZ_Real.values[20:] - tangXZ_Virt.values[20:]),2))) / len(tangXZ_Real.values[20:])

                        print('Past if statement: ', i)

                        MSE = np.round(np.sum(np.power(np.abs(tangXZ_Real.values[20:] - tangXZ_Virt.values[20:]),2)) / len(tangXZ_Real.values[20:]),4) * 100 # Convert to cm
                        endPosError = (np.round(np.nanmean(np.abs(tangXZ_Virt.values[-1-30:-1] - tangXZ_Real.values[-1-30:-1])),3)/30) * 100 # Convert to cm
                        posError = np.round(np.nanmean(np.abs(tangXZ_Virt.values[20:] - tangXZ_Real.values[20:])),3) * 100 # Convert to cm
                        velError = np.round(np.nanmean(np.abs(tangXZ_Virt_Vel[20:] - tangXZ_Real_Vel[20:])),3) # Convert to cm

                        # print('Mean Square Error: ', MSE, 'cm')
                        # print('Target Hit Error: ', endPosError, 'cm')
                        # print('Average Positional Error: ', posError, 'cm')
                        # print('Velocity Error: ', velError, 'ms-1')

                        dat_List.append([lag, MSE, endPosError, posError, velError, times[-1]])



                except Exception as e:
                    print('MY_ERROR: Size of array: ', len(tangXZ_Real))
                    zeroArray = np.tile(np.zeros, (1, 100))
                    tangXZ_Real_Vel = zeroArray
                    tangXZ_Virt_Vel = zeroArray

            except Exception as ex:
                print('My_Error_2: ', ex)
    
        
        df_tmp = pd.DataFrame(dat_List, columns =['Lag' ,'MSE_Error', 'Hit_Error', 'AvPos_Error', 'Vel_Error', 'Time'])
        df_tmp.insert(0, 'PtxID', ptx, True)

        if df_out is None:
            df_out = df_tmp
        else:
            df_out =df_out.loc[:,~df_out.columns.duplicated()]
            df_out = pd.concat((df_out, df_tmp))            
        # times = np.arange(0,)

    df_out =df_out.loc[:,~df_out.columns.duplicated()]
    return df_out

In [ ]:
df_bmp160_low_errors = ComputeErrors2(df_bmp160_low)
df_bmp160_mid_errors = ComputeErrors2(df_bmp160_mid)
df_bmp160_high_errors = ComputeErrors2(df_bmp160_high)

df_bmp120_low_errors = ComputeErrors2(df_bmp120_low)
df_bmp120_mid_errors = ComputeErrors2(df_bmp120_mid)
df_bmp120_high_errors = ComputeErrors2(df_bmp120_high)

df_bmp80_low_errors = ComputeErrors2(df_bmp80_low)
df_bmp80_mid_errors = ComputeErrors2(df_bmp80_mid)
df_bmp80_high_errors = ComputeErrors2(df_bmp80_high)

In [ ]:
print('Processed ptx: ', pd.unique(df_bmp160_low['PtxID']))
print('Original ptx: ', pd.unique(df_all['PtxID']))


In [ ]:
df_bmp160_mid_errors
# df_bmp80_low_errors

In [ ]:
def AverageSTE(dataz):
    ste = np.nanstd(dataz) / np.sqrt(len(dataz))
    avr = np.nanmean(dataz)

    return avr,ste


In [ ]:

# sns.displot(df_bmp160_mid_errors, x="Lag", binwidth=10)
xVals = np.arange(len(df_bmp160_high_errors.Lag))
# g = sns.relplot(x=xVals, y="Lag", hue="Hit_Error", data=df_bmp160_mid_errors)
g = sns.relplot(x=xVals, y="Hit_Error", size="Lag", sizes=(10, 200), data=df_bmp160_high_errors)

In [ ]:
bmp120_mid_errorsArr

In [ ]:
# MSE, endPosError, posError, velError
bmp80_low_errorsArr = np.array(df_bmp80_low_errors)
bmp120_mid_errorsArr = np.array(df_bmp120_mid_errors)
bmp160_high_errorsArr = np.array(df_bmp160_high_errors)

plt.figure()
# plt.plot(bmp80_low_errorsArr[:,0],'r-o')
plt.plot(bmp120_mid_errorsArr[:,1],'g-o')
plt.plot(bmp160_high_errorsArr[:,2],'b-o')
plt.legend(['End PosErr','Path Error'])
plt.ylim([-0.5, 100])
plt.title('Error Between Quest and MoCap')
plt.xlabel('Trial Number')
plt.ylabel('Error / cm')

# plt.figure()
# av,se = AverageSTE(bmp120_mid_errorsArr[:,0])
# plt.errorbar([0,1,2], [av, np.nan, np.nan], [np.nan, np.nan, se],ms=15,color='r')

# plt.legend(['MSE','End PosErr','Average Error'])

plt.savefig(str(np.round(time.time())) + '_Errors_EndPointPath.png', dpi=600, bbox_inches='tight')

In [ ]:
# # Set theme
# sns.set_style('whitegrid')
 
# # Violin plot
# sns.violinplot(x='xPos', y='zPos', data=df)

In [ ]:
df_bmp160_mid_errors

In [ ]:

trials = np.arange(0, len(df_bmp80_low_errors))
df_bmp80_low_errors.insert(0, 'TrialNum', trials)

trials = np.arange(0, len(df_bmp120_low_errors))
df_bmp120_low_errors.insert(0, 'TrialNum', trials)

trials = np.arange(0, len(df_bmp160_low_errors))
df_bmp160_low_errors.insert(0, 'TrialNum', trials)

trials = np.arange(0, len(df_bmp80_mid_errors))
df_bmp80_mid_errors.insert(0, 'TrialNum', trials)
trials = np.arange(0, len(df_bmp120_mid_errors))
df_bmp120_mid_errors.insert(0, 'TrialNum', trials)
trials = np.arange(0, len(df_bmp160_mid_errors))
df_bmp160_mid_errors.insert(0, 'TrialNum', trials)

trials = np.arange(0, len(df_bmp80_high_errors))
df_bmp80_high_errors.insert(0, 'TrialNum', trials)
trials = np.arange(0, len(df_bmp120_high_errors))
df_bmp120_high_errors.insert(0, 'TrialNum', trials)
trials = np.arange(0, len(df_bmp160_high_errors))
df_bmp160_high_errors.insert(0, 'TrialNum', trials)

df_bmp160_high_errors

In [ ]:
df_bmp160_high_errors

In [ ]:
# Density Plot
# sns.kdeplot(df.xPos, df.zPos)
fig, axs = plt.subplots(3, 3, figsize=(12, 5), sharey=True)
fig.suptitle('AvPos_Error / cm')

axs[0, 0].set_title('Low 80 BPM')
sns.distplot(df_bmp80_low_errors.AvPos_Error, ax=axs[0,0])

axs[0, 1].set_title('LOW 120 BPM')
sns.distplot(df_bmp120_low_errors.AvPos_Error, ax=axs[0,1])

axs[0, 2].set_title('LOW 160 BPM')
sns.distplot(df_bmp160_low_errors.AvPos_Error, ax=axs[0,2])

axs[1, 0].set_title('MID 80 BPM')
sns.distplot(df_bmp80_mid_errors.AvPos_Error, ax=axs[1,0])

axs[1, 1].set_title('MID 120 BPM')
sns.distplot(df_bmp120_mid_errors.AvPos_Error, ax=axs[1,1])

axs[1, 2].set_title('MID 160 BPM')
sns.distplot(df_bmp160_mid_errors.AvPos_Error, ax=axs[1,2])

axs[2, 0].set_title('HIGH 80 BPM')
sns.distplot(df_bmp80_high_errors.AvPos_Error, ax=axs[2,0])

axs[2, 1].set_title('HIGH 120 BPM')
sns.distplot(df_bmp120_high_errors.AvPos_Error, ax=axs[2,1])

axs[2, 2].set_title('HIGH 160 BPM')
sns.distplot(df_bmp160_high_errors.AvPos_Error, ax=axs[2,2])

plt.figure()
plt.title('HIGH 160 BPM')
sns.distplot(df_bmp160_high_errors.AvPos_Error)


## Create master metric table

In [ ]:
df_bmp80_low_errors

In [ ]:
df_all_errors = None

df_bmp80_low_errors.insert(0, 'height', 0, True)
df_bmp80_low_errors.insert(0, 'tempo', 80, True)
df_all_errors = df_bmp80_low_errors 

df_bmp120_low_errors.insert(0, 'height', 0, True)
df_bmp120_low_errors.insert(0, 'tempo', 120, True)
df_all_errors =  df = pd.concat((df_all_errors, df_bmp120_low_errors))

df_bmp160_low_errors.insert(0, 'height', 0, True)
df_bmp160_low_errors.insert(0, 'tempo', 160, True)
df_all_errors =  df = pd.concat((df_all_errors, df_bmp160_low_errors))

df_bmp80_mid_errors.insert(0, 'height', 1, True)
df_bmp80_mid_errors.insert(0, 'tempo', 80, True)
df_all_errors =  df = pd.concat((df_all_errors, df_bmp80_mid_errors))

df_bmp120_mid_errors.insert(0, 'height', 1, True)
df_bmp120_mid_errors.insert(0, 'tempo', 120, True)
df_all_errors =  df = pd.concat((df_all_errors, df_bmp120_mid_errors))

df_bmp160_mid_errors.insert(0, 'height', 2, True)
df_bmp160_mid_errors.insert(0, 'tempo', 160, True)
df_all_errors =  df = pd.concat((df_all_errors, df_bmp160_mid_errors))

df_bmp80_high_errors.insert(0, 'height', 2, True)
df_bmp80_high_errors.insert(0, 'tempo', 80, True)
df_all_errors =  df = pd.concat((df_all_errors, df_bmp80_high_errors))

df_bmp120_high_errors.insert(0, 'height', 2, True)
df_bmp120_high_errors.insert(0, 'tempo', 120, True)
df_all_errors =  df = pd.concat((df_all_errors, df_bmp120_high_errors))

df_bmp160_high_errors.insert(0, 'height', 2, True)
df_bmp160_high_errors.insert(0, 'tempo', 160, True)
df_all_errors =  df = pd.concat((df_all_errors, df_bmp160_high_errors))

df_all_errors

# TODO
- [ ] Incorporate targets into the analysis
- [ ] Cleanup the notebook 

In [ ]:
# sns.regplot(x='TrialNum', y='Lag', x_bins=4, data=df_bmp120_high_errors, x_estimator=np.mean)
# sns.violinplot(x='TrialNum', y='Lag', data=df_bmp120_high_errors)
# sns.regplot(x="TrialNum", y="Vel_Error", data=df_bmp120_high_errors,x_estimator=np.mean, logx=True)

# sns.relplot(x="TrialNum", y="MSE_Error", kind="line", data=df_bmp120_high_errors)

# sns.set_theme(style="whitegrid")
# sns.violinplot(data=df_bmp120_high_errors, x="Vel_Error", y="MSE_Error", hue="Lag", split=False, inner="quart", linewidth=1) #,palette={"Yes": "b", "No": ".85"})
# sns.despine(left=True)

sns.set_theme(style="darkgrid")
cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)
g = sns.relplot(
    data=df_all_errors,
    x="TrialNum", y="Hit_Error", hue="height", size = "tempo", sizes=(75, 225), palette="Blues",alpha=0.85
)
plt.ylim([-0.25, 5])
plt.xlabel('Trial Number')
plt.ylabel('Positional Error / cm')
plt.title('Path Error Between Quest and MoCap')

plt.savefig(str(np.round(time.time())) + 'Path Error.png', dpi=600, bbox_inches='tight')

In [ ]:
df_all_errors.head(2)

In [ ]:
plt.figure()

# g1 = sns.barplot(x='tempo', y="height", palette="Reds", data=df_all_errors, estimator=np.nanmean)
# # plt.ylim([0, 0.15])
# plt.xlabel('Tempo')

ptxMeanErrTH = df_all_errors.groupby(['PtxID', 'tempo', 'height'])['Hit_Error'].mean()
df_Tempo_Height = ptxMeanErrTH.to_frame()


sns.scatterplot(x = 'tempo', y='height', size='Hit_Error', sizes=(100, 400), alpha=.5, data=df_Tempo_Height)

plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.title('Average Hit Error / m',fontsize=14)
plt.xlabel('Tempo (bpm)')
plt.xticks([80,120,160],['Slow','Medium','Fast'])
plt.ylabel('Height')
plt.yticks([0,1,2],['Low','Mid','Heigh'])

In [ ]:
plt.figure()
plt.subplot(221)
g1 = sns.barplot(x='tempo', y="Hit_Error", palette="Reds", data=df_all_errors, estimator=np.nanmean)
plt.ylim([0, 0.15])
plt.xlabel('Tempo')

plt.subplot(222)
g2 = sns.barplot(x='tempo', y="AvPos_Error", palette="Reds", data=df_all_errors, estimator=np.nanmean)
plt.ylim([0, 10])
plt.xlabel('Tempo')

plt.subplot(223)
g1 = sns.barplot(x='height', y="Hit_Error", palette="Blues", data=df_all_errors, estimator=np.nanmean)
plt.ylim([0, 0.15])
plt.xlabel('Height')

plt.subplot(224)
g2 = sns.barplot(x='height', y="AvPos_Error", palette="Blues", data=df_all_errors, estimator=np.nanmean)
plt.ylim([0, 10])
plt.xlabel('Height')

# plt.title('Average \n' + 'metric' + "\n \n")
# plt.ylabel('metric' + " / Degrees") #
# plt.xlabel('Phases')

In [ ]:
ax = sns.barplot(x="height", y="AvPos_Error", palette="Greens", data=df_all_errors, estimator=np.nanmean)
plt.title('Cummulative Path Error \n Between Quest and MoCap \n \n')
plt.ylabel('Positional Error / cm')
plt.xlabel('Height')
plt.xticks([0,1,2],['Low','Mid','High'])

plt.savefig(str(np.round(time.time())) + 'Cummulative Error Height.png', dpi=600, bbox_inches='tight')

statannot.add_stat_annotation(
    ax,
    data=df_all_errors,
    x='tempo',
    y='AvPos_Error',
    hue='height',
#     box_pairs=[("80", '80'), ('80','80'), ('80', '80')],
    test="t-test_ind",
    text_format="star",
    loc="outside",
)

In [ ]:
lowMask = (df_all_errors['height'] == 0) & (df_all_errors['tempo'] == 120)
midMask = (df_all_errors['height'] == 1) & (df_all_errors['tempo'] == 120)
highMask = (df_all_errors['height'] == 2) & (df_all_errors['tempo'] == 120)

sci.ttest_ind(df_all_errors[midMask]['AvPos_Error'] , df_all_errors[highMask]['AvPos_Error'])

In [ ]:
# import scipy.stats as sci 

# PlotErrorBars2(bmaxZVels, 'r')
# PlotErrorBars2(amaxZVels, 'g')

# print('baseline: ', np.shape(bmaxZVels), 'adaptation: ', np.shape(amaxZVels))
# h = sci.ttest_ind(bmaxZVels, amaxZVels)
# print('ttest: ', h)

# plt.xticks(ticks = [0], labels=['Z'])
# plt.ylabel('Max Velocity ms-1')
# plt.xlabel('Axis')
# plt.title('Average Max Velocity')
# plt.legend(['Baseline','Adaptation'])

In [ ]:
# # Reshape and average standard errors:
# print('Arr: ', vRow_A_point_z)

# # Sort indeces according to target id:
# targetSortedIdx = np.argsort(target)
# targets = np.take(target,targetSortedIdx)

# vrow_A_point_z = np.take(vRow_A_point_z, targetSortedIdx)

# print('\n target: ', target, ' targets: ', targets)

# xVals = np.linspace(0,len(targets)-1, len(targets))
# print('Size of X vals: ', np.shape(xVals), ' Vals: ', np.shape(vrow_A_point_z), ' SE: ', np.shape(ste_z))

# # Real finger tip 
# plt.figure()
# plt.subplot(121)
# plt.errorbar(xVals, vrow_A_point_z, vste_z, color = 'b', ms=8, alpha=0.5)
# plt.plot(vrow_A_point_z,'ko',ms=8,alpha=1.0)
# plt.xticks(xVals, targets, rotation=45)
# plt.ylim([0, 0.6])
# plt.xlim([-1, 6])
# plt.xlabel('Target ID')
# plt.ylabel('Virtual Finger Forward Position / m')
# plt.title(p + ' Virtual')

# # Real finger tip 
# row_A_point_z = np.take(Row_A_point_z,targetSortedIdx)
# targets = np.take(target,targetSortedIdx)

# plt.subplot(122)
# plt.errorbar(xVals, row_A_point_z, ste_z, color = 'b', ms=8, alpha=0.5)
# plt.plot(row_A_point_z,'ko',ms=8,alpha=1.0)
# plt.xticks(xVals, targets, rotation=45)
# plt.ylim([0, 0.6])
# plt.xlim([-1, 6])
# plt.xlabel('Target ID Row: ' + trow)
# plt.ylabel('Real Finger Forward Position / m')
# plt.title(p + ' Real')


In [ ]:
#     try:
#         if len(vste_x) > 24:
#             vste_x = np.delete(vste_x, -1)
#         if len(vste_z) > 24:
#             vste_z = np.delete(vste_z, -1)
#         if len(ste_x) > 24:
#             ste_x = np.delete(ste_x, -1)
#         if len(ste_z) > 24:
#             ste_z = np.delete(ste_z, -1)
#     except Exception as e:
#         vste_x = np.zeros(6)
#         vste_z = np.zeros(6)
#         ste_x = np.zeros(6)
#         ste_z = np.zeros(6)
# 
#     vste_x = np.nanmean(np.reshape(vste_x, (4, 6)), axis=0)
#     vste_z = np.nanmean(np.reshape(vste_z, (4, 6)), axis=0)
#     ste_x = np.nanmean(np.reshape(ste_x, (4, 6)), axis=0)
#     ste_z = np.nanmean(np.reshape(ste_z, (4, 6)), axis=0)

In [ ]:
# df_motions = None
# np.set_printoptions(suppress=True)
# newArrLength = 100

# # mask = (df_all['PtxID'] == 'Susan') & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == 'low') & (df_all['tempos'] == '80') & (df_all['TrialNum'] == '42')
# ptxIds = pd.unique(df_all['PtxID'])
# print('Participants: ', ptxIds)
# # Ptx number: 
# # 0 = Susan
# # 1 = Davide
# # 2 = Joe
# # 3 = Poppy
# # 4 = Katrina
# # 5 = Max
# # 6 = Pete

# includedPtxs = [0,1,3,4,6]

# ptxNum = 2 
# tempores = 'noTempo'
# heightes = 'low'

# trials = np.arange(0,72)

# # for partici in includedPtxs:

# for trial in trials:
    
#     print('Trial number: ', trial)
    
#     try:
#         mask = (df_all['TrialNum'] == str(trial)) & (df_all['PtxID'] == ptxIds[ptxNum]) & (df_all['tempos'] == tempores) & (df_all['gameObjectName'] == 'realFingerTip') & (df_all['height'] == heightes)
#         mask_virt = (df_all['TrialNum'] == str(trial)) & (df_all['PtxID'] == ptxIds[ptxNum]) & (df_all['tempos'] == tempores) & (df_all['gameObjectName'] == 'r_index_fingernail_marker') & (df_all['height'] == heightes)

#         # Compute actual sampling rate
#         timetaken = df_all[mask]['time'].values
#         timetaken2 = ResizeArray(timetaken, newArrLength)
#         timetaken3 = np.round(timetaken2,1)
#         timetaken3 = timetaken3.tolist()
#         indexOfStart = timetaken3.index(0.0) # indexOfStart = np.where(timetaken == 0.0)
#         samplingRate = np.round(1.0 / ((len(timetaken3)-indexOfStart) / timetaken3[-1]), 4)
#         print('Sampling Rate: ', np.round(1.0/samplingRate))
        
#         # Get individual velocities for real hand ---------------------------------------
#         pos_x = ResizeArray(df_all[mask]['xPos'].values, newArrLength)
#         pos_xf = savgol_filter(pos_x, 21, 9)
#         vel_x = np.gradient(pos_xf / samplingRate)

#         pos_y = ResizeArray(df_all[mask]['yPos'].values, newArrLength)
#         pos_yf = savgol_filter(pos_y, 21, 9)
#         vel_y = np.gradient(pos_yf / samplingRate)

#         pos_z = ResizeArray(df_all[mask]['zPos'].values, newArrLength)
#         pos_zf = savgol_filter(pos_z, 21, 9)
#         vel_z = np.gradient(pos_zf / samplingRate)
#         vel_type_1 = np.sqrt(np.power(vel_x,2) + np.power(vel_y,2) + np.power(vel_z,2))
#         vel_type_1f = savgol_filter(vel_type_1, 21, 9)

#         # Get individual velocities for virtual hand ---------------------------------------
#         pos_xv = ResizeArray(df_all[mask_virt]['xPos'].values, newArrLength)
#         pos_xfv = savgol_filter(pos_xv, 21, 9)
#         vel_xv = np.gradient(pos_xfv / samplingRate)

#         pos_yv = ResizeArray(df_all[mask_virt]['yPos'].values, newArrLength)
#         pos_yfv = savgol_filter(pos_yv, 21, 9)
#         vel_yv = np.gradient(pos_yfv / samplingRate)

#         pos_zv = ResizeArray(df_all[mask_virt]['zPos'].values, newArrLength)
#         pos_zfv = savgol_filter(pos_zv, 21, 9)
#         vel_zv = np.gradient(pos_zfv / samplingRate)
        
#         vel_type_1v = np.sqrt(np.power(vel_xv,2) + np.power(vel_yv,2) + np.power(vel_zv,2))
#         vel_type_1fv = savgol_filter(vel_type_1v, 21, 9)
#         # ------------------------------------------------------------------------------
        
#         pos_tx = df_all[mask]['xTPos'].values
#         pos_ty = df_all[mask]['yTPos'].values
#         pos_tz = df_all[mask]['zTPos'].values

#         # Pos
#         plt.figure()
#         plt.subplot(1,3,1)
#         plt.plot(pos_x, pos_z,'k-o')
#         plt.plot(pos_xv, pos_zv,'m-o')
#         plt.plot(pos_tx, pos_tz,'r-o',ms=10)
#         plt.plot([pos_x, pos_xv], [pos_z, pos_zv],'b-',alpha=0.5) # Connection line between trajectories to indicate delays
#         plt.legend(['XZ Motion','XZ Virtual','Target'])
        
#         # Last positional data point in different colours for visualisation purposes 
#         plt.subplot(1,3,3)
#         plt.plot(pos_x[-1-5:-1], pos_z[-1-5:-1],'k-o', ms=8)
#         plt.plot(pos_xv[-1-5:-1], pos_zv[-1-5:-1],'m-o', ms=6)
#         plt.plot([pos_x[-1], pos_xv[-1]], [pos_z[-1], pos_zv[-1]], 'r-o', ms=15, alpha=0.5)
        
# #         plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# #         plt.axis('equal')
    
#         # Vel
#         plt.subplot(1,3,2)
#         plt.plot(vel_type_1,'r-')
#         plt.plot(vel_type_1f,'g-')
#         plt.plot(vel_type_1fv,'m-')
#         startIdx = indexOfStart
#         plt.plot([startIdx,startIdx],[0,2.75],'k--',linewidth=3)
#         plt.title('Vel_Type_1')
#         plt.legend(['Raw','Filtered','Filt Virt','Start'])
# #         plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# #         plt.axis('equal')
#         plt.ylim([-0.1, 5])
        
#         # Velocity around start of trial 
#         realVel = vel_type_1f[startIdx-15:startIdx+15]
#         virtVel = vel_type_1fv[startIdx-15:startIdx+15]
        
#         lag = CrossCorr(realVel, virtVel, samplingRate)
#         print('Lag: ', np.round(lag, 2), ' ms')
        
#         maxVel = np.max(realVel)
#         maxVelVirt = np.max(virtVel)
#         print('Max vel: \n', 'Real: ', np.round(maxVel,2),'\n', 'Virt: ', np.round(maxVelVirt,2))
        
#         real = np.asarray([pos_x,pos_z])
#         virt = np.asarray([pos_xv,pos_zv])
        
#         manhattanErr = np.sum(np.abs(real-virt)) / len(pos_x)
        
#         ns = 5
#         p1m = [np.nanmean(pos_x[-1-ns:-1]), np.nanmean(pos_z[-1-ns:-1])]
#         p2m = [np.nanmean(pos_xv[-1-ns:-1]), np.nanmean(pos_zv[-1-ns:-1])]
#         distance2 = np.round(math.sqrt( ((p1m[0]-p2m[0])**2)+((p1m[1]-p2m[1])**2) ) * 100,2)
        
#         print('Per sample path offset (Manhattan err): ', np.round(manhattanErr * 100,2), ' cm')
#         print('End point err: ', distance2, ' cm')
        
#         plt.tight_layout()
#         plt.show()
        
#     except Exception as e:
#         print('Err: ', e)
